<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_Mask2Former_Evaluador_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Sistema de Evaluación Avanzado Mask2Former
==============================================================

Framework completo para evaluación sistemática de modelos Mask2Former
con análisis de características avanzadas usando librerías especializadas.

Características principales:
- Análisis geométrico con Shapely
- Características de textura con Mahotas y Scikit-image
- Exportación dual: JSON propio + formato COCO
- Sistema de logging estructurado
- Organización automática de directorios

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña
Fecha: 2025
"""


'\nSistema de Evaluación Avanzado Mask2Former\n==============================================================\n\nFramework completo para evaluación sistemática de modelos Mask2Former\ncon análisis de características avanzadas usando librerías especializadas.\n\nCaracterísticas principales:\n- Análisis geométrico con Shapely\n- Características de textura con Mahotas y Scikit-image\n- Exportación dual: JSON propio + formato COCO\n- Sistema de logging estructurado\n- Organización automática de directorios\n\nAutor: Jesús L.\nProyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación\nUniversidad: Universidad Oberta de Cataluña\nFecha: 2025\n'

In [3]:
# Librerías principales que necesitas instalar
!pip install shapely
!pip install mahotas
!pip install scikit-image

# Librerías adicionales que podrían no estar actualizadas
!pip install opencv-python
!pip install psutil

# Si tienes problemas con versiones específicas, usa estas versiones estables:
!pip install shapely>=1.8.0
!pip install mahotas>=1.4.0
!pip install scikit-image>=0.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.6 MB/s eta 0:00:00


In [103]:
import torch
import numpy as np
import cv2
import json
import time
import os
import psutil
import logging
import hashlib
import warnings
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Dict, List, Any, Optional, Tuple

# Librerías especializadas para análisis avanzado
import mahotas as mh
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
from shapely import affinity
from skimage import measure, morphology, feature, filters, segmentation
from skimage.color import rgb2gray, rgb2hsv, rgb2lab
from sklearn.cluster import KMeans

from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configuración para entorno Colab
from google.colab import drive
drive.mount('/content/drive')

# Suprimir warnings para salida más limpia
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
# =============================================================================
# CONFIGURACIÓN Y ESTRUCTURAS DE DATOS
# =============================================================================

@dataclass
class ModeloInfo:
    """Información completa de un modelo Mask2Former"""
    nombre_hf: str
    nombre_corto: str
    tipo: str  # 'instancia' o 'semantico'
    dataset: str  # 'COCO', 'ADE20K', etc.
    arquitectura: str  # 'Swin-Large', 'Swin-Base', etc.
    descripcion: str

@dataclass
class ConfiguracionUmbrales:
    """Configuración de umbrales con metadatos"""
    nombre: str
    valores: List[float]
    descripcion: str

@dataclass
class ConfigEvaluacion:
    """Configuración centralizada del sistema de evaluación avanzado"""

    # Rutas base
    BASE_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former")
    DATASET_PATH: Path = BASE_PATH / "imagenes"

    # Parámetros de procesamiento
    MAX_SIZE_IMAGEN: int = 1024
    MAX_IMAGENES_LOTE: int = 50
    GUARDAR_VISUALIZACIONES: bool = True
    LIMPIAR_CACHE_CADA: int = 25
    GENERAR_FORMATO_COCO: bool = True

    # Modelos disponibles
    MODELOS: List[ModeloInfo] = None

    # Configuraciones de umbrales
    UMBRALES: Dict[str, ConfiguracionUmbrales] = None

    def __post_init__(self):
        if self.MODELOS is None:
            self.MODELOS = [
                ModeloInfo(
                    nombre_hf="facebook/mask2former-swin-large-coco-instance",
                    nombre_corto="swin-large-coco-instance",
                    tipo='instancia',
                    dataset='COCO',
                    arquitectura='Swin-Large',
                    descripcion='Segmentación por instancia - Mayor precisión'
                ),
                ModeloInfo(
                    nombre_hf="facebook/mask2former-swin-base-ade-semantic",
                    nombre_corto="swin-base-ade-semantic",
                    tipo='semantico',
                    dataset='ADE20K',
                    arquitectura='Swin-Base',
                    descripcion='Segmentación semántica - Balance eficiencia/precisión'
                ),
                ModeloInfo(
                    nombre_hf="facebook/mask2former-swin-small-coco-instance",
                    nombre_corto="swin-small-coco-instance",
                    tipo='instancia',
                    dataset='COCO',
                    arquitectura='Swin-Small',
                    descripcion='Segmentación por instancia - Más rápido'
                )
            ]

        if self.UMBRALES is None:
            self.UMBRALES = {
                'ultra_sensible': ConfiguracionUmbrales(
                    nombre='ultra_sensible',
                    valores=[0.0001, 0.001, 0.01, 0.1],
                    descripcion='Detecta cambios mínimos - Máxima sensibilidad'
                ),
                'alta_sensibilidad': ConfiguracionUmbrales(
                    nombre='alta_sensibilidad',
                    valores=[0.001, 0.01, 0.05, 0.1, 0.3],
                    descripcion='Sensibilidad alta para detección temprana'
                ),
                'sensibilidad_media': ConfiguracionUmbrales(
                    nombre='sensibilidad_media',
                    valores=[0.01, 0.1, 0.3, 0.5],
                    descripcion='Balance entre precisión y recall'
                ),
                'baja_sensibilidad': ConfiguracionUmbrales(
                    nombre='baja_sensibilidad',
                    valores=[0.3, 0.5, 0.7],
                    descripcion='Solo detecciones muy confiables'
                )
            }

    def crear_directorio_ejecucion(self) -> Path:
        """Crea directorio único para cada ejecución con timestamp"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        directorio_ejecucion = self.BASE_PATH / "resultados" / f"ejecucion_{timestamp}"

        subdirectorios = [
            "logs",
            "resultados_json",
            "formato_coco",
            "caracteristicas_avanzadas",
            "visualizaciones",
            "resumenes"
        ]

        for subdir in subdirectorios:
            (directorio_ejecucion / subdir).mkdir(parents=True, exist_ok=True)

        return directorio_ejecucion

In [105]:
# =============================================================================
# SISTEMA DE LOGGING ESTRUCTURADO
# =============================================================================

class LoggerManager:
    """Gestor centralizado de logging para diferentes componentes del sistema"""

    def __init__(self, directorio_logs: Path):
        self.directorio_logs = directorio_logs
        self.loggers = {}
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )

    def crear_logger(self, nombre: str, archivo_log: str = None) -> logging.Logger:
        """Crea un logger específico para un componente"""
        if nombre in self.loggers:
            return self.loggers[nombre]

        logger = logging.getLogger(nombre)
        logger.setLevel(logging.INFO)

        if logger.handlers:
            logger.handlers.clear()

        if archivo_log is None:
            archivo_log = f"{nombre.lower()}_{self.timestamp}.log"

        archivo_path = self.directorio_logs / archivo_log
        handler_archivo = logging.FileHandler(archivo_path, encoding='utf-8')

        formatter = logging.Formatter(
            '%(asctime)s - %(levelname)-8s - %(message)s',
            datefmt='%H:%M:%S'
        )
        handler_archivo.setFormatter(formatter)

        logger.addHandler(handler_archivo)
        self.loggers[nombre] = logger

        return logger

In [107]:
# =============================================================================
# ANALIZADOR DE MÁSCARAS CON SHAPELY
# =============================================================================

class MaskAnalyzer:
    """Analizador especializado de máscaras de segmentación usando Shapely"""

    def __init__(self):
        self.epsilon_factor = 0.02

    def analizar_mascaras_completo(self, masks: np.ndarray, scores: List[float],
                                  imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis completo de todas las máscaras detectadas"""
        if len(masks) == 0:
            return self._resultado_vacio()

        resultado = {
            'metadatos': {
                'num_mascaras': len(masks),
                'imagen_shape': imagen_shape,
                'total_pixels': imagen_shape[0] * imagen_shape[1]
            },
            'mascaras_individuales': [],
            'analisis_conjunto': {},
            'estadisticas_globales': {}
        }

        polygons_validos = []

        # Análisis individual de cada máscara
        for i, (mask, score) in enumerate(zip(masks, scores)):
            analisis_individual = self._analizar_mascara_individual(
                mask, score, i, imagen_shape
            )
            resultado['mascaras_individuales'].append(analisis_individual)

            if analisis_individual['shapely_geometry']['polygon_valido']:
                polygons_validos.append(analisis_individual['shapely_geometry']['polygon'])

        # Análisis conjunto
        if polygons_validos:
            resultado['analisis_conjunto'] = self._analizar_conjunto_mascaras(
                polygons_validos, imagen_shape
            )

        # Estadísticas globales
        resultado['estadisticas_globales'] = self._calcular_estadisticas_globales(
            resultado['mascaras_individuales']
        )

        return resultado

    def _analizar_mascara_individual(self, mask: np.ndarray, score: float,
                                   indice: int, imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis completo de una máscara individual"""
        mask_binary = (mask > 0.5).astype(np.uint8)

        resultado = {
            'indice': indice,
            'score_confianza': float(score),
            'caracteristicas_geometricas': {},
            'caracteristicas_forma': {},
            'caracteristicas_contextuales': {},
            'shapely_geometry': {}
        }

        try:
            resultado['caracteristicas_geometricas'] = self._extraer_geometricas_basicas(
                mask_binary, imagen_shape
            )
            resultado['shapely_geometry'] = self._analizar_con_shapely(mask_binary)
            resultado['caracteristicas_forma'] = self._extraer_forma_calidad(mask_binary)
            resultado['caracteristicas_contextuales'] = self._extraer_contextuales(
                mask_binary, imagen_shape
            )
        except Exception as e:
            resultado['error'] = str(e)

        return resultado

    def _extraer_geometricas_basicas(self, mask: np.ndarray,
                                   imagen_shape: Tuple[int, int]) -> Dict[str, float]:
        """Extrae características geométricas básicas usando scikit-image"""
        props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

        if props is None:
            return self._geometricas_vacias()

        total_pixels = imagen_shape[0] * imagen_shape[1]

        return {
            'area_pixels': float(props.area),
            'area_percentage': float(props.area / total_pixels * 100),
            'perimeter': float(props.perimeter),
            'compactness': float(4 * np.pi * props.area / (props.perimeter ** 2)) if props.perimeter > 0 else 0.0,
            'aspect_ratio': float(props.major_axis_length / props.minor_axis_length) if props.minor_axis_length > 0 else 0.0,
            'orientation_angle': float(np.degrees(props.orientation)),
            'centroid_y': float(props.centroid[0]),
            'centroid_x': float(props.centroid[1]),
            'solidity': float(props.solidity),
            'extent': float(props.extent),
            'eccentricity': float(props.eccentricity),
            'major_axis_length': float(props.major_axis_length),
            'minor_axis_length': float(props.minor_axis_length)
        }

    def _analizar_con_shapely(self, mask: np.ndarray) -> Dict[str, Any]:
        """Análisis geométrico avanzado usando Shapely"""
        try:
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if not contours:
                return self._shapely_vacio()

            contour = max(contours, key=cv2.contourArea)
            epsilon = self.epsilon_factor * cv2.arcLength(contour, True)
            contour_simplified = cv2.approxPolyDP(contour, epsilon, True)

            if len(contour_simplified) < 3:
                return self._shapely_vacio()

            coords = [(point[0][0], point[0][1]) for point in contour_simplified]
            if coords[0] != coords[-1]:
                coords.append(coords[0])

            polygon = Polygon(coords)

            if not polygon.is_valid:
                polygon = polygon.buffer(0)

            if not polygon.is_valid:
                return self._shapely_vacio()

            convex_hull = polygon.convex_hull

            return {
                'polygon_valido': True,
                'polygon': polygon,
                'area_shapely': float(polygon.area),
                'perimeter_shapely': float(polygon.length),
                'convex_hull_area': float(convex_hull.area),
                'convexity_ratio': float(polygon.area / convex_hull.area),
                'is_simple': bool(polygon.is_simple),
                'num_vertices': len(coords) - 1,
                'bounds': polygon.bounds,
                'centroid_shapely': (float(polygon.centroid.x), float(polygon.centroid.y))
            }

        except Exception as e:
            return {
                'polygon_valido': False,
                'error_shapely': str(e),
                **self._shapely_vacio()
            }

    def _extraer_forma_calidad(self, mask: np.ndarray) -> Dict[str, float]:
        """Características de forma y calidad de la máscara"""
        try:
            labeled_mask = measure.label(mask)
            num_components = labeled_mask.max()

            edges = cv2.Canny(mask.astype(np.uint8) * 255, 50, 150)
            edge_pixels = np.sum(edges > 0)

            grad_x = cv2.Sobel(mask.astype(np.float32), cv2.CV_32F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(mask.astype(np.float32), cv2.CV_32F, 0, 1, ksize=3)
            gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
            edge_smoothness = float(np.std(gradient_magnitude[gradient_magnitude > 0]))

            filled = morphology.binary_fill_holes(mask)
            holes_area = np.sum(filled) - np.sum(mask)

            if np.sum(mask) > 0:
                equivalent_diameter = np.sqrt(4 * np.sum(mask) / np.pi)
                ideal_perimeter = np.pi * equivalent_diameter
                actual_perimeter = float(measure.perimeter(mask))
                roughness = actual_perimeter / ideal_perimeter if ideal_perimeter > 0 else 0.0
            else:
                roughness = 0.0

            return {
                'num_components': int(num_components),
                'edge_pixels': int(edge_pixels),
                'edge_smoothness': edge_smoothness,
                'holes_area': int(holes_area),
                'holes_percentage': float(holes_area / np.sum(filled) * 100) if np.sum(filled) > 0 else 0.0,
                'roughness_factor': roughness,
                'is_single_component': bool(num_components == 1)
            }

        except Exception as e:
            return {
                'error_forma': str(e),
                'num_components': 0,
                'edge_pixels': 0,
                'edge_smoothness': 0.0,
                'holes_area': 0,
                'holes_percentage': 0.0,
                'roughness_factor': 0.0,
                'is_single_component': False
            }

    def _extraer_contextuales(self, mask: np.ndarray,
                            imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Características contextuales de posición y distribución"""
        h, w = imagen_shape
        props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

        if props is None:
            return self._contextuales_vacios()

        cy, cx = props.centroid
        pos_y_rel = cy / h
        pos_x_rel = cx / w

        position_region = self._clasificar_posicion(pos_x_rel, pos_y_rel)

        dist_top = cy
        dist_bottom = h - cy
        dist_left = cx
        dist_right = w - cx

        center_x, center_y = w / 2, h / 2
        dist_to_center = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
        dist_to_center_normalized = dist_to_center / np.sqrt(center_x**2 + center_y**2)

        bbox = props.bbox
        bbox_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        bbox_coverage = float(bbox_area / (h * w))

        return {
            'centroid_relative': {'x': float(pos_x_rel), 'y': float(pos_y_rel)},
            'position_region': position_region,
            'distances_to_edges': {
                'top': float(dist_top),
                'bottom': float(dist_bottom),
                'left': float(dist_left),
                'right': float(dist_right)
            },
            'distance_to_center': float(dist_to_center),
            'distance_to_center_normalized': float(dist_to_center_normalized),
            'bbox_coverage_percentage': float(bbox_coverage * 100),
            'aspect_vs_image': float((bbox[3] - bbox[1]) / (bbox[2] - bbox[0])) if (bbox[2] - bbox[0]) > 0 else 0.0
        }

    def _analizar_conjunto_mascaras(self, polygons: List[Polygon],
                                  imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis de relaciones entre múltiples máscaras"""
        if len(polygons) < 2:
            return {'num_mascaras': len(polygons), 'analisis_conjunto': 'insuficientes_mascaras'}

        union_polygon = unary_union(polygons)
        overlaps = []
        distances = []

        for i in range(len(polygons)):
            for j in range(i + 1, len(polygons)):
                poly1, poly2 = polygons[i], polygons[j]

                intersection = poly1.intersection(poly2)
                overlap_area = intersection.area if hasattr(intersection, 'area') else 0
                overlap_percentage = (overlap_area / min(poly1.area, poly2.area)) * 100
                overlaps.append(overlap_percentage)

                distance = poly1.distance(poly2)
                distances.append(distance)

        centroids = [poly.centroid for poly in polygons]
        centroid_distances = []

        for i in range(len(centroids)):
            for j in range(i + 1, len(centroids)):
                dist = centroids[i].distance(centroids[j])
                centroid_distances.append(dist)

        return {
            'num_mascaras': len(polygons),
            'area_total_union': float(union_polygon.area),
            'coverage_percentage': float(union_polygon.area / (imagen_shape[0] * imagen_shape[1]) * 100),
            'overlaps': {
                'mean_overlap_percentage': float(np.mean(overlaps)),
                'max_overlap_percentage': float(np.max(overlaps)),
                'num_overlapping_pairs': int(np.sum(np.array(overlaps) > 0))
            },
            'distances': {
                'mean_distance': float(np.mean(distances)),
                'min_distance': float(np.min(distances)),
                'max_distance': float(np.max(distances))
            },
            'spatial_distribution': {
                'mean_centroid_distance': float(np.mean(centroid_distances)),
                'centroid_spread': float(np.std(centroid_distances))
            }
        }

    def _calcular_estadisticas_globales(self, mascaras_individuales: List[Dict]) -> Dict[str, Any]:
        """Estadísticas globales del conjunto de máscaras"""
        if not mascaras_individuales:
            return {}

        areas = []
        compactness = []
        scores = []
        roughness = []

        for mask_data in mascaras_individuales:
            if 'error' not in mask_data:
                geom = mask_data.get('caracteristicas_geometricas', {})
                forma = mask_data.get('caracteristicas_forma', {})

                areas.append(geom.get('area_percentage', 0))
                compactness.append(geom.get('compactness', 0))
                scores.append(mask_data.get('score_confianza', 0))
                roughness.append(forma.get('roughness_factor', 0))

        def safe_stats(values):
            if not values:
                return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'areas_estadisticas': safe_stats(areas),
            'compactness_estadisticas': safe_stats(compactness),
            'scores_estadisticas': safe_stats(scores),
            'roughness_estadisticas': safe_stats(roughness),
            'mascaras_exitosas': len([m for m in mascaras_individuales if 'error' not in m])
        }

    # Métodos auxiliares
    def _resultado_vacio(self):
        return {
            'metadatos': {'num_mascaras': 0},
            'mascaras_individuales': [],
            'analisis_conjunto': {},
            'estadisticas_globales': {}
        }

    def _geometricas_vacias(self):
        return {k: 0.0 for k in ['area_pixels', 'area_percentage', 'perimeter',
                                'compactness', 'aspect_ratio', 'orientation_angle',
                                'centroid_y', 'centroid_x', 'solidity', 'extent',
                                'eccentricity', 'major_axis_length', 'minor_axis_length']}

    def _shapely_vacio(self):
        return {
            'polygon_valido': False,
            'polygon': None,
            'area_shapely': 0.0,
            'perimeter_shapely': 0.0,
            'convex_hull_area': 0.0,
            'convexity_ratio': 0.0,
            'is_simple': False,
            'num_vertices': 0,
            'bounds': (0, 0, 0, 0),
            'centroid_shapely': (0.0, 0.0)
        }

    def _contextuales_vacios(self):
        return {
            'centroid_relative': {'x': 0.0, 'y': 0.0},
            'position_region': 'desconocida',
            'distances_to_edges': {'top': 0.0, 'bottom': 0.0, 'left': 0.0, 'right': 0.0},
            'distance_to_center': 0.0,
            'distance_to_center_normalized': 0.0,
            'bbox_coverage_percentage': 0.0,
            'aspect_vs_image': 0.0
        }

    def _clasificar_posicion(self, x_rel: float, y_rel: float) -> str:
        """Clasifica la posición de la máscara en la imagen"""
        if y_rel < 0.33:
            if x_rel < 0.33:
                return 'superior_izquierda'
            elif x_rel < 0.67:
                return 'superior_centro'
            else:
                return 'superior_derecha'
        elif y_rel < 0.67:
            if x_rel < 0.33:
                return 'centro_izquierda'
            elif x_rel < 0.67:
                return 'centro'
            else:
                return 'centro_derecha'
        else:
            if x_rel < 0.33:
                return 'inferior_izquierda'
            elif x_rel < 0.67:
                return 'inferior_centro'
            else:
                return 'inferior_derecha'

In [108]:
# =============================================================================
# EXTRACTOR DE CARACTERÍSTICAS AVANZADAS
# =============================================================================

class ExtractorCaracteristicasAvanzado:
    """Extractor de características usando librerías especializadas"""

    def __init__(self):
        self.logger = None

    def set_logger(self, logger: logging.Logger):
        """Asigna logger para el extractor"""
        self.logger = logger

    def analizar_imagen_completa(self, imagen: Image.Image, ruta: str) -> Dict[str, Any]:
        """Análisis completo usando librerías especializadas"""
        img_array = np.array(imagen)
        img_gray = rgb2gray(img_array)
        img_hsv = rgb2hsv(img_array)
        img_lab = rgb2lab(img_array)

        img_gray_uint8 = (img_gray * 255).astype(np.uint8)

        caracteristicas = {
            'metadatos_basicos': self._extraer_metadatos_basicos(imagen, ruta),
            'color_y_paleta': self._analizar_color_avanzado(img_array, img_hsv, img_lab),
            'texturas_mahotas': self._extraer_texturas_mahotas(img_gray_uint8),
            'texturas_skimage': self._extraer_texturas_skimage(img_gray),
            'caracteristicas_geometricas': self._analizar_geometria_avanzada(img_gray_uint8),
            'multiscale_features': self._extraer_multiscale_features(img_gray),
            'propiedades_regionales': self._analizar_propiedades_regionales(img_gray),
            'descriptores_locales': self._extraer_descriptores_locales(img_gray_uint8)
        }

        return caracteristicas

    def _extraer_metadatos_basicos(self, imagen: Image.Image, ruta: str) -> Dict:
        """Metadatos básicos de la imagen"""
        w, h = imagen.size
        return {
            'dimensiones': {'ancho': w, 'alto': h},
            'aspecto_ratio': round(w / h, 3),
            'megapixeles': round((w * h) / 1000000, 2),
            'orientacion': 'horizontal' if w > h else 'vertical' if h > w else 'cuadrada',
            'formato': ruta.split('.')[-1].lower() if '.' in ruta else 'desconocido'
        }

    def _analizar_color_avanzado(self, img_rgb: np.ndarray, img_hsv: np.ndarray, img_lab: np.ndarray) -> Dict:
        """Análisis de color usando sklearn para paleta dominante"""
        pixels = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
        kmeans.fit(pixels)

        colores_dominantes = kmeans.cluster_centers_.astype(int).tolist()
        proporciones = np.bincount(kmeans.labels_) / len(kmeans.labels_)

        return {
            'paleta_dominante': colores_dominantes,
            'proporciones_colores': proporciones.tolist(),
            'estadisticas_rgb': {
                'media': np.mean(img_rgb, axis=(0,1)).tolist(),
                'std': np.std(img_rgb, axis=(0,1)).tolist(),
                'rango': {
                    'min': np.min(img_rgb, axis=(0,1)).tolist(),
                    'max': np.max(img_rgb, axis=(0,1)).tolist()
                }
            },
            'hsv_global': {
                'hue_medio': float(np.mean(img_hsv[:,:,0])),
                'saturacion_media': float(np.mean(img_hsv[:,:,1])),
                'valor_medio': float(np.mean(img_hsv[:,:,2]))
            },
            'lab_luminancia': {
                'L_medio': float(np.mean(img_lab[:,:,0])),
                'a_medio': float(np.mean(img_lab[:,:,1])),
                'b_medio': float(np.mean(img_lab[:,:,2]))
            }
        }

    def _extraer_texturas_mahotas(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Mahotas"""
        try:
            haralick_features = mh.features.haralick(img_gray, return_mean=True)
            lbp = mh.features.lbp(img_gray, radius=1, points=8, ignore_zeros=False)

            try:
                zernike_features = mh.features.zernike_moments(img_gray, radius=21)
            except:
                zernike_features = np.zeros(25)

            otsu_threshold = mh.otsu(img_gray)
            pftas = mh.features.pftas(img_gray)

            return {
                'haralick_features': haralick_features.tolist(),
                'lbp_histogram': np.histogram(lbp, bins=50)[0].tolist(),
                'zernike_moments': zernike_features.tolist(),
                'otsu_threshold': float(otsu_threshold),
                'pftas': pftas.tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en Mahotas: {str(e)}")
            return {
                'error': f"Error en Mahotas: {str(e)}",
                'haralick_features': [],
                'lbp_histogram': [],
                'zernike_moments': [],
                'otsu_threshold': 0.0,
                'pftas': []
            }

    def _extraer_texturas_skimage(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Scikit-image"""
        try:
            lbp_skimage = feature.local_binary_pattern(img_gray, P=8, R=1, method='uniform')

            img_scaled = (img_gray * 255).astype(np.uint8)
            glcm = feature.graycomatrix(img_scaled, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                      levels=256, symmetric=True, normed=True)

            contrast = feature.graycoprops(glcm, 'contrast').mean()
            dissimilarity = feature.graycoprops(glcm, 'dissimilarity').mean()
            homogeneity = feature.graycoprops(glcm, 'homogeneity').mean()
            energy = feature.graycoprops(glcm, 'energy').mean()
            correlation = feature.graycoprops(glcm, 'correlation').mean()

            return {
                'lbp_uniform_histogram': np.histogram(lbp_skimage, bins=10)[0].tolist(),
                'glcm_properties': {
                    'contrast': float(contrast),
                    'dissimilarity': float(dissimilarity),
                    'homogeneity': float(homogeneity),
                    'energy': float(energy),
                    'correlation': float(correlation)
                }
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en Scikit-image: {str(e)}")
            return {
                'error': f"Error en Scikit-image: {str(e)}",
                'lbp_uniform_histogram': [],
                'glcm_properties': {}
            }

    def _analizar_geometria_avanzada(self, img_gray: np.ndarray) -> Dict:
        """Análisis geométrico usando mahotas y scikit-image"""
        try:
            edges_canny = feature.canny(img_gray / 255.0)
            edges_sobel = mh.sobel(img_gray)

            corners = feature.corner_harris(img_gray / 255.0)
            corner_peaks = feature.corner_peaks(corners, min_distance=5)

            moments = measure.moments(img_gray)
            centroid = measure.centroid(img_gray)

            return {
                'bordes_canny': float(np.sum(edges_canny)),
                'bordes_sobel_intensidad': float(np.mean(edges_sobel)),
                'num_corners': len(corner_peaks),
                'centroide': [float(centroid[0]), float(centroid[1])],
                'momentos_hu': measure.moments_hu(moments).tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en análisis geométrico: {str(e)}")
            return {
                'error': f"Error en análisis geométrico: {str(e)}",
                'bordes_canny': 0.0,
                'bordes_sobel_intensidad': 0.0,
                'num_corners': 0,
                'centroide': [0.0, 0.0],
                'momentos_hu': []
            }

    def _extraer_multiscale_features(self, img_gray: np.ndarray) -> Dict:
        """Características multi-escala usando scikit-image"""
        try:
            features_multiscale = feature.multiscale_basic_features(
                img_gray,
                intensity=True,
                edges=True,
                texture=True,
                sigma_min=0.5,
                sigma_max=8
            )

            return {
                'num_features': features_multiscale.shape[-1],
                'feature_means': np.mean(features_multiscale, axis=(0,1)).tolist(),
                'feature_stds': np.std(features_multiscale, axis=(0,1)).tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en features multiscale: {str(e)}")
            return {
                'error': f"Error en features multiscale: {str(e)}",
                'num_features': 0,
                'feature_means': [],
                'feature_stds': []
            }

    def _analizar_propiedades_regionales(self, img_gray: np.ndarray) -> Dict:
        """Análisis de propiedades regionales usando segmentación"""
        try:
            # Especificar explícitamente que es imagen en escala de grises
            segments = segmentation.slic(img_gray, n_segments=100, compactness=10, channel_axis=None)
            regions = measure.regionprops(segments, intensity_image=img_gray)

            if regions:
                areas = [r.area for r in regions]
                eccentricities = [r.eccentricity for r in regions]
                intensities = [r.mean_intensity for r in regions]

                return {
                    'num_regiones': len(regions),
                    'area_promedio': float(np.mean(areas)),
                    'excentricidad_promedio': float(np.mean(eccentricities)),
                    'intensidad_promedio_regiones': float(np.mean(intensities)),
                    'variabilidad_areas': float(np.std(areas)),
                    'variabilidad_intensidades': float(np.std(intensities))
                }
            else:
                return {'num_regiones': 0}

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en análisis regional: {str(e)}")
            return {
                'error': f"Error en análisis regional: {str(e)}",
                'num_regiones': 0
            }

    def _extraer_descriptores_locales(self, img_gray: np.ndarray) -> Dict:
        """Descriptores locales usando OpenCV"""
        try:
            img_cv = img_gray.copy()

            orb = cv2.ORB_create(nfeatures=100)
            keypoints_orb = orb.detect(img_cv, None)

            fast = cv2.FastFeatureDetector_create()
            keypoints_fast = fast.detect(img_cv, None)

            return {
                'orb_keypoints': len(keypoints_orb),
                'fast_keypoints': len(keypoints_fast),
                'keypoint_density': (len(keypoints_orb) + len(keypoints_fast)) / (img_gray.shape[0] * img_gray.shape[1])
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en descriptores locales: {str(e)}")
            return {
                'error': f"Error en descriptores locales: {str(e)}",
                'orb_keypoints': 0,
                'fast_keypoints': 0,
                'keypoint_density': 0.0
            }

In [109]:
class GeneradorVisualizaciones:
    """Genera visualizaciones comparativas imagen original vs detecciones"""

    def __init__(self, directorio_salida: Path, logger: logging.Logger = None):
        self.directorio_salida = directorio_salida
        self.logger = logger

    def generar_visualizacion_completa(self, imagen_original: Image.Image,
                                     nombre_archivo: str, resultados_deteccion: Dict,
                                     umbral_principal: float = None,
                                     modelo_nombre: str = "modelo") -> Optional[str]:
        """
        Genera visualización completa: original + máscaras + overlay

        Args:
            imagen_original: Imagen PIL original
            nombre_archivo: Nombre del archivo de imagen
            resultados_deteccion: Resultados de detección del modelo
            umbral_principal: Umbral a usar para visualización (usa el primero si es None)
            modelo_nombre: Nombre del modelo para el archivo

        Returns:
            Ruta del archivo de visualización generado
        """
        try:
            # Seleccionar umbral para visualización
            detecciones_umbral = resultados_deteccion.get('detecciones_por_umbral', {})
            if not detecciones_umbral:
                return None

            if umbral_principal is None:
                # Usar el primer umbral disponible
                umbral_key = list(detecciones_umbral.keys())[0]
            else:
                umbral_key = f'umbral_{umbral_principal}'

            if umbral_key not in detecciones_umbral:
                return None

            datos_umbral = detecciones_umbral[umbral_key]

            # Crear figura con subplots
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            fig.suptitle(f'Detección de Personas - {nombre_archivo}', fontsize=16, fontweight='bold')

            # 1. Imagen original
            axes[0].imshow(imagen_original)
            axes[0].set_title('Imagen Original', fontsize=14)
            axes[0].axis('off')

            # 2. Máscaras detectadas
            self._generar_visualizacion_mascaras(axes[1], imagen_original, datos_umbral, umbral_key)

            # 3. Overlay (original + máscaras superpuestas)
            self._generar_overlay(axes[2], imagen_original, datos_umbral, umbral_key)

            # Información adicional
            personas = datos_umbral.get('personas_detectadas', 0)
            total = datos_umbral.get('total_detecciones', 0)
            umbral_val = umbral_key.replace('umbral_', '')

            fig.text(0.5, 0.02, f'Personas detectadas: {personas} | Total detecciones: {total} | Umbral: {umbral_val}',
                    ha='center', fontsize=12, style='italic')

            # Guardar visualización
            umbral_str = f"umbral_{umbral_principal:.4f}".replace('.', '')
            nombre_vis = f"vis_{modelo_nombre}_{umbral_str}_{nombre_archivo.replace('.jpg', '').replace('.png', '').replace('.jpeg', '')}.png"
            archivo_salida = self.directorio_salida / nombre_vis

            plt.tight_layout()
            plt.savefig(archivo_salida, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close()

            if self.logger:
                self.logger.info(f"Visualización guardada: {nombre_vis}")

            return str(archivo_salida)

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error generando visualización para {nombre_archivo}: {str(e)}")
            return None

    def _generar_visualizacion_mascaras(self, ax, imagen_original: Image.Image,
                                       datos_umbral: Dict, umbral_key: str):
        """Genera visualización de máscaras detectadas"""

        # Crear imagen base en escala de grises
        img_gray = np.array(imagen_original.convert('L'))
        h, w = img_gray.shape
        mask_combined = np.zeros_like(img_gray, dtype=np.float32)

        # Intentar usar máscaras reales guardadas
        mascaras_raw = datos_umbral.get('mascaras_raw')
        num_mascaras = 0

        if mascaras_raw and len(mascaras_raw) > 0:
            # Usar máscaras reales de Mask2Former
            for i, mask in enumerate(mascaras_raw):
                if mask is not None and mask.size > 0:
                    # Convertir máscara a binario
                    mask_binary = (mask > 0.5).astype(np.float32)

                    # Redimensionar si es necesario
                    if mask_binary.shape != (h, w):
                        mask_binary = cv2.resize(mask_binary, (w, h), interpolation=cv2.INTER_NEAREST)

                    # Agregar con diferentes intensidades para cada máscara
                    mask_combined += mask_binary * (0.3 + (i * 0.2))
                    num_mascaras += 1

        # Si no hay máscaras raw, intentar desde análisis Shapely
        if num_mascaras == 0:
            analisis_mascaras = datos_umbral.get('analisis_mascaras_avanzado')
            if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
                for i, mascara_data in enumerate(analisis_mascaras['mascaras_individuales']):
                    if 'error' not in mascara_data:
                        geom = mascara_data.get('caracteristicas_geometricas', {})

                        # Crear máscara desde bbox si está disponible
                        if all(key in geom for key in ['bbox_min_col', 'bbox_min_row', 'bbox_max_col', 'bbox_max_row']):
                            x1 = int(geom['bbox_min_col'])
                            y1 = int(geom['bbox_min_row'])
                            x2 = int(geom['bbox_max_col'])
                            y2 = int(geom['bbox_max_row'])

                            # Crear máscara rectangular básica
                            if x2 > x1 and y2 > y1 and x2 <= w and y2 <= h:
                                mask_region = np.zeros((h, w))
                                mask_region[y1:y2, x1:x2] = (i + 1) * 0.3
                                mask_combined += mask_region
                                num_mascaras += 1

        # Si aún no hay máscaras, crear visualización dummy
        if num_mascaras == 0:
            personas = datos_umbral.get('personas_detectadas', 0)
            if personas > 0:
                # Crear máscara dummy en el centro
                center_h, center_w = h // 2, w // 2
                size = min(h, w) // 4
                mask_combined[center_h-size:center_h+size, center_w-size:center_w+size] = 0.5
                num_mascaras = personas

        # Visualizar máscaras
        if num_mascaras > 0:
            # Mostrar imagen base + máscaras coloreadas
            ax.imshow(img_gray, cmap='gray', alpha=0.7)
            mask_colored = np.ma.masked_where(mask_combined == 0, mask_combined)
            ax.imshow(mask_colored, cmap='Reds', alpha=0.8, vmin=0, vmax=1)
            ax.set_title(f'Máscaras Detectadas ({num_mascaras})', fontsize=14)
        else:
            ax.imshow(img_gray, cmap='gray')
            ax.set_title('No se detectaron personas', fontsize=14)

        ax.axis('off')

    def _generar_overlay(self, ax, imagen_original: Image.Image,
                        datos_umbral: Dict, umbral_key: str):
        """Genera overlay de imagen original con detecciones superpuestas"""

        img_array = np.array(imagen_original)

        # Usar máscaras reales si están disponibles
        mascaras_raw = datos_umbral.get('mascaras_raw')
        analisis_mascaras = datos_umbral.get('analisis_mascaras_avanzado')

        contornos_dibujados = 0

        # Intentar dibujar contornos de máscaras reales
        if mascaras_raw and len(mascaras_raw) > 0:
            for i, mask in enumerate(mascaras_raw):
                if mask is not None and mask.size > 0:
                    # Convertir máscara a binario
                    mask_binary = (mask > 0.5).astype(np.uint8)

                    # Redimensionar si es necesario
                    h, w = img_array.shape[:2]
                    if mask_binary.shape != (h, w):
                        mask_binary = cv2.resize(mask_binary, (w, h), interpolation=cv2.INTER_NEAREST)

                    # Encontrar contornos
                    contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    for contour in contours:
                        if cv2.contourArea(contour) > 100:  # Filtrar contornos muy pequeños
                            # Dibujar contorno
                            from matplotlib.patches import Polygon as MplPolygon
                            contour_points = contour.reshape(-1, 2)
                            polygon = MplPolygon(contour_points, linewidth=2,
                                               edgecolor='red', facecolor='red', alpha=0.3)
                            ax.add_patch(polygon)

                            # Añadir etiqueta
                            x, y, w_bbox, h_bbox = cv2.boundingRect(contour)
                            score = datos_umbral.get('confianza_scores', [1.0])[min(i, len(datos_umbral.get('confianza_scores', [1.0]))-1)]
                            ax.text(x, y-5, f'Persona {i+1}\nScore: {score:.3f}',
                                   bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8),
                                   fontsize=9, verticalalignment='top')
                            contornos_dibujados += 1

        # Si no hay máscaras raw, usar bounding boxes desde análisis Shapely
        if contornos_dibujados == 0 and analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
            for i, mascara_data in enumerate(analisis_mascaras['mascaras_individuales']):
                if 'error' not in mascara_data:
                    geom = mascara_data.get('caracteristicas_geometricas', {})
                    score = mascara_data.get('score_confianza', 0.0)

                    # Dibujar bounding box
                    if all(key in geom for key in ['bbox_min_col', 'bbox_min_row', 'bbox_max_col', 'bbox_max_row']):
                        x1 = int(geom['bbox_min_col'])
                        y1 = int(geom['bbox_min_row'])
                        x2 = int(geom['bbox_max_col'])
                        y2 = int(geom['bbox_max_row'])

                        # Dibujar rectángulo
                        from matplotlib.patches import Rectangle
                        rect = Rectangle((x1, y1), x2-x1, y2-y1,
                                       linewidth=3, edgecolor='red', facecolor='none')
                        ax.add_patch(rect)

                        # Añadir texto con score
                        ax.text(x1, y1-5, f'Persona {i+1}\nScore: {score:.3f}',
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
                               fontsize=10, verticalalignment='top')
                        contornos_dibujados += 1

        ax.imshow(img_array)
        personas = datos_umbral.get('personas_detectadas', 0)
        ax.set_title(f'Overlay - {personas} Persona(s) - {contornos_dibujados} Visualizadas', fontsize=14)
        ax.axis('off')

    def generar_resumen_visual(self, resultados_procesamiento: List[Dict],
                              nombre_modelo: str) -> Optional[str]:
        """Genera un resumen visual con múltiples imágenes procesadas"""
        try:
            # Filtrar resultados exitosos
            exitosos = [r for r in resultados_procesamiento if r.get('metadatos', {}).get('exitoso', False)]
            if len(exitosos) == 0:
                return None

            # Tomar una muestra representativa (máximo 12 imágenes)
            muestra = exitosos[:12] if len(exitosos) > 12 else exitosos

            # Calcular grid
            n_imgs = len(muestra)
            cols = min(4, n_imgs)
            rows = (n_imgs + cols - 1) // cols

            fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
            fig.suptitle(f'Resumen Visual - {nombre_modelo}', fontsize=16, fontweight='bold')

            # MANEJO CORREGIDO DE AXES
            if n_imgs == 1:
                axes_list = [axes]
            elif rows == 1:
                axes_list = axes if hasattr(axes, '__len__') else [axes]
            elif cols == 1:
                axes_list = axes if hasattr(axes, '__len__') else [axes]
            else:
                axes_list = axes.flatten()

            for i, resultado in enumerate(muestra):
                ax = axes_list[i]

                # Información de la imagen
                info_img = resultado['imagen']
                nombre_archivo = info_img['archivo']

                # Estadísticas de detección (primer umbral)
                detecciones = resultado['deteccion'].get('detecciones_por_umbral', {})
                if detecciones:
                    primer_umbral = list(detecciones.values())[0]
                    personas = primer_umbral.get('personas_detectadas', 0)
                    score_max = primer_umbral.get('score_maximo', 0)

                    # Color según detección
                    color = 'green' if personas > 0 else 'gray'

                    ax.text(0.5, 0.7, nombre_archivo, ha='center', va='center',
                           fontsize=10, weight='bold', transform=ax.transAxes)
                    ax.text(0.5, 0.5, f'Personas: {personas}', ha='center', va='center',
                           fontsize=12, color=color, weight='bold', transform=ax.transAxes)
                    ax.text(0.5, 0.3, f'Score máx: {score_max:.3f}', ha='center', va='center',
                           fontsize=10, transform=ax.transAxes)

                    ax.set_facecolor('lightgreen' if personas > 0 else 'lightgray')
                else:
                    ax.text(0.5, 0.5, f'{nombre_archivo}\nError de procesamiento',
                           ha='center', va='center', fontsize=10, transform=ax.transAxes)
                    ax.set_facecolor('lightcoral')

                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_aspect('equal')

            # Ocultar axes sobrantes
            for i in range(n_imgs, len(axes_list)):
                axes_list[i].set_visible(False)

            # Estadísticas generales
            total_personas = sum(
                list(r['deteccion']['detecciones_por_umbral'].values())[0].get('personas_detectadas', 0)
                for r in exitosos if r['deteccion'].get('detecciones_por_umbral')
            )

            imagenes_con_personas = sum(
                1 for r in exitosos
                if r['deteccion'].get('detecciones_por_umbral') and
                list(r['deteccion']['detecciones_por_umbral'].values())[0].get('personas_detectadas', 0) > 0
            )

            fig.text(0.5, 0.02,
                    f'Total procesadas: {len(exitosos)} | Con personas: {imagenes_con_personas} | Total personas: {total_personas}',
                    ha='center', fontsize=12, style='italic')

            # Guardar resumen
            nombre_resumen = f"resumen_visual_{nombre_modelo}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
            archivo_resumen = self.directorio_salida / nombre_resumen

            plt.tight_layout()
            plt.savefig(archivo_resumen, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close()

            if self.logger:
                self.logger.info(f"Resumen visual guardado: {nombre_resumen}")

            return str(archivo_resumen)

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error generando resumen visual: {str(e)}")
            return None

In [110]:
# =============================================================================
# EXPORTADOR FORMATO COCO
# =============================================================================

class ExportadorCOCO:
    """Exportador de resultados a formato COCO"""

    def __init__(self, logger: logging.Logger = None):
        self.logger = logger
        self.coco_data = {
            "info": {
                "description": "Mask2Former Evaluation Results",
                "version": "1.0",
                "year": datetime.now().year,
                "contributor": "TFM Evaluation Framework",
                "date_created": datetime.now().isoformat()
            },
            "images": [],
            "annotations": [],
            "categories": [
                {
                    "id": 1,
                    "name": "person",
                    "supercategory": "person"
                }
            ]
        }
        self.image_id = 1
        self.annotation_id = 1

    def agregar_imagen(self, nombre_archivo: str, ancho: int, alto: int) -> int:
        """Agrega una imagen al dataset COCO"""
        imagen_info = {
            "id": self.image_id,
            "file_name": nombre_archivo,
            "width": ancho,
            "height": alto
        }
        self.coco_data["images"].append(imagen_info)

        current_id = self.image_id
        self.image_id += 1
        return current_id

    def agregar_anotacion(self, image_id: int, mascara: np.ndarray, score: float, bbox: List[int]):
        """Agrega una anotación de segmentación al formato COCO"""
        try:
            # Convertir máscara a RLE (Run Length Encoding) simplificado
            # En un sistema completo usarías pycocotools, aquí una versión simplificada
            area = int(np.sum(mascara > 0.5))

            if area == 0:
                return

            anotacion = {
                "id": self.annotation_id,
                "image_id": image_id,
                "category_id": 1,  # person
                "bbox": bbox,  # [x, y, width, height]
                "area": area,
                "iscrowd": 0,
                "score": float(score)
            }

            self.coco_data["annotations"].append(anotacion)
            self.annotation_id += 1

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error agregando anotación COCO: {str(e)}")

    def exportar(self, archivo_salida: Path) -> None:
        """Exporta los datos en formato COCO"""
        try:
            with open(archivo_salida, 'w', encoding='utf-8') as f:
                json.dump(self.coco_data, f, indent=2, ensure_ascii=False)

            if self.logger:
                self.logger.info(f"Archivo COCO exportado: {archivo_salida.name}")
                self.logger.info(f"Imágenes: {len(self.coco_data['images'])}, Anotaciones: {len(self.coco_data['annotations'])}")

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error exportando COCO: {str(e)}")

In [111]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utils:
    """Funciones utilitarias reutilizables"""

    @staticmethod
    def cargar_imagenes(ruta: str, extensiones: Tuple[str, ...] = (".jpg", ".png", ".jpeg")) -> List[str]:
        """Carga todas las imágenes desde una ruta recursivamente"""
        path = Path(ruta)
        imagenes = [
            str(p) for p in path.glob("**/*")
            if p.suffix.lower() in extensiones and p.is_file()
        ]
        return sorted(imagenes)

    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> Image.Image:
        """Prepara una imagen para procesamiento"""
        try:
            img = Image.open(ruta).convert("RGB")
            if max(img.size) > max_size:
                img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
            return img
        except Exception as e:
            raise ValueError(f"Error cargando imagen {ruta}: {str(e)}")

    @staticmethod
    def calcular_hash_imagen(ruta: str) -> str:
        """Calcula hash MD5 para identificación única de imagen"""
        try:
            with open(ruta, 'rb') as f:
                return hashlib.md5(f.read()).hexdigest()[:12]
        except Exception:
            return "hash_error"

    @staticmethod
    def guardar_json(datos: Any, archivo: Path, indent: int = 2) -> None:
        """Guarda datos en formato JSON con manejo de errores"""
        try:
            with open(archivo, 'w', encoding='utf-8') as f:
                json.dump(datos, f, indent=indent, ensure_ascii=False, default=str)
        except Exception as e:
            raise IOError(f"Error guardando JSON en {archivo}: {str(e)}")

    @staticmethod
    def crear_nombre_archivo(modelo_info: ModeloInfo, config_umbral: str, timestamp: str) -> str:
        """Crea nombres de archivo descriptivos y únicos"""
        return f"mask2former_{modelo_info.tipo}_{modelo_info.dataset.lower()}_{modelo_info.arquitectura.lower().replace('-', '')}_{config_umbral}_{timestamp}.json"

In [112]:
# =============================================================================
# DETECTOR DE PERSONAS
# =============================================================================

class DetectorPersonas:
    """Detector de personas usando modelos Mask2Former con análisis avanzado"""

    def __init__(self, modelo_info: ModeloInfo, logger: logging.Logger,
                 extractor: ExtractorCaracteristicasAvanzado, mask_analyzer: MaskAnalyzer):
        self.modelo_info = modelo_info
        self.logger = logger
        self.extractor = extractor
        self.mask_analyzer = mask_analyzer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.logger.info("="*60)
        self.logger.info(f"🤖 INICIALIZANDO MODELO: {modelo_info.nombre_corto}")
        self.logger.info(f"📌 Tipo: {modelo_info.tipo.upper()}")
        self.logger.info(f"🏗️ Arquitectura: {modelo_info.arquitectura}")
        self.logger.info(f"📊 Dataset: {modelo_info.dataset}")
        self.logger.info(f"💻 Dispositivo: {self.device}")

        try:
            self.logger.info("⏳ Cargando procesador de imágenes...")
            self.processor = AutoImageProcessor.from_pretrained(modelo_info.nombre_hf)

            self.logger.info("⏳ Cargando modelo de segmentación...")
            self.model = AutoModelForUniversalSegmentation.from_pretrained(modelo_info.nombre_hf)

            self.model.to(self.device)
            self.model.eval()

            self.es_semantico = (modelo_info.tipo == 'semantico')
            self._configurar_clases()

            self.logger.info("✅ Modelo cargado exitosamente")
            self.logger.info("="*60)

        except Exception as e:
            self.logger.error(f"❌ Error cargando modelo: {str(e)}")
            raise

    def _configurar_clases(self) -> None:
        """Configura las clases y encuentra la clase 'persona'"""
        if hasattr(self.model.config, 'id2label'):
            self.id2label = self.model.config.id2label
            self.logger.info(f"🏷️ Clases disponibles: {len(self.id2label)}")

            self.clase_persona = 0
            for clase_id, nombre in self.id2label.items():
                if any(term in nombre.lower() for term in ['person', 'people', 'human']):
                    self.clase_persona = clase_id
                    self.logger.info(f"👤 Clase persona encontrada: ID {clase_id} = '{nombre}'")
                    break
            else:
                self.logger.warning(f"⚠️ Usando clase por defecto (ID 0) como 'persona'")
        else:
            self.clase_persona = 0
            self.logger.warning(f"⚠️ Sin diccionario de clases, usando ID 0 como 'persona'")

    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float]) -> Dict[str, Any]:
        """Ejecuta detección completa con análisis avanzado"""
        inicio_tiempo = time.time()
        memoria_inicial = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

        try:
            w, h = imagen.size
            inputs = self.processor(images=imagen, return_tensors="pt").to(self.device)

            # Inferencia
            inicio_inferencia = time.time()
            with torch.no_grad():
                outputs = self.model(**inputs)
            tiempo_inferencia_ms = (time.time() - inicio_inferencia) * 1000

            # Métricas de memoria
            memoria_maxima = torch.cuda.max_memory_allocated() if torch.cuda.is_available() else 0
            memoria_usada_mb = (memoria_maxima - memoria_inicial) / (1024 ** 2)

            resultados = {
                'timestamp': datetime.now().isoformat(),
                'modelo_info': asdict(self.modelo_info),
                'rendimiento': {
                    'tiempo_inferencia_ms': tiempo_inferencia_ms,
                    'memoria_usada_mb': memoria_usada_mb,
                    'fps_estimado': 1000 / tiempo_inferencia_ms
                }
            }

            # Post-procesamiento específico por tipo
            if self.es_semantico:
                resultados.update(self._procesar_segmentacion_semantica(outputs, (h, w), umbrales))
            else:
                resultados.update(self._procesar_segmentacion_instancia(outputs, (h, w), umbrales))

            tiempo_total_ms = (time.time() - inicio_tiempo) * 1000
            resultados['rendimiento']['tiempo_total_ms'] = tiempo_total_ms

            return resultados

        except Exception as e:
            self.logger.error(f"Error en detección: {str(e)}")
            return self._crear_resultado_error(str(e), umbrales)

    def _procesar_segmentacion_semantica(self, outputs, target_size: Tuple[int, int], umbrales: List[float]) -> Dict:
        """Procesa resultados de segmentación semántica con análisis avanzado"""
        try:
            resultado_semantico = self.processor.post_process_semantic_segmentation(
                outputs, target_sizes=[target_size]
            )[0]

            unique_classes = torch.unique(resultado_semantico)
            total_pixels = resultado_semantico.numel()

            resultados_umbrales = {}

            for umbral in umbrales:
                persona_mask = (resultado_semantico == self.clase_persona)
                persona_pixels = persona_mask.sum().item()
                porcentaje_persona = (persona_pixels / total_pixels) * 100

                personas_detectadas = 1 if porcentaje_persona >= (umbral * 100) and persona_pixels > 50 else 0

                # Análisis avanzado de la máscara si hay detección
                analisis_mascaras = None
                if personas_detectadas > 0:
                    mask_np = persona_mask.cpu().numpy().astype(np.float32)
                    analisis_mascaras = self.mask_analyzer.analizar_mascaras_completo(
                        [mask_np], [1.0], target_size
                    )

                resultados_umbrales[f'umbral_{umbral}'] = {
                    'personas_detectadas': personas_detectadas,
                    'total_clases_detectadas': len(unique_classes),
                    'clases_presentes': unique_classes.tolist(),
                    'estadisticas_persona': {
                        'pixels_persona': persona_pixels,
                        'porcentaje_imagen': round(porcentaje_persona, 3),
                        'umbral_aplicado': umbral,
                        'criterio_cumplido': personas_detectadas > 0
                    },
                    'analisis_mascaras_avanzado': analisis_mascaras,
                    'confianza_scores': [1.0] if personas_detectadas > 0 else [],
                    'score_maximo': 1.0 if len(unique_classes) > 0 else 0.0
                }

            return {'detecciones_por_umbral': resultados_umbrales}

        except Exception as e:
            self.logger.error(f"Error en segmentación semántica: {str(e)}")
            return self._crear_resultado_error(str(e), umbrales)

    def _procesar_segmentacion_instancia(self, outputs, target_size: Tuple[int, int], umbrales: List[float]) -> Dict:
        """Procesa resultados de segmentación de instancia con análisis avanzado"""
        resultados_umbrales = {}

        for umbral in umbrales:
            try:
                resultado = self.processor.post_process_instance_segmentation(
                    outputs, target_sizes=[target_size], threshold=umbral
                )[0]

                labels = resultado.get("labels", torch.tensor([]))
                scores = resultado.get("scores", torch.tensor([]))
                masks = resultado.get("masks", torch.tensor([]))

                # Filtrar detecciones de personas
                indices_personas = [i for i, l in enumerate(labels) if int(l.item()) == self.clase_persona]
                personas_detectadas = len(indices_personas)

                scores_personas = [float(scores[i].item()) for i in indices_personas]

                # Análisis avanzado de máscaras de personas
                analisis_mascaras = None
                if personas_detectadas > 0 and len(masks) > 0:
                    masks_personas = [masks[i].cpu().numpy() for i in indices_personas]
                    scores_personas_mask = [scores[i].item() for i in indices_personas]

                    analisis_mascaras = self.mask_analyzer.analizar_mascaras_completo(
                        masks_personas, scores_personas_mask, target_size
                    )

                todas_clases = [int(l.item()) for l in labels]
                todos_scores = [float(s.item()) for s in scores]

                resultados_umbrales[f'umbral_{umbral}'] = {
                    'personas_detectadas': personas_detectadas,
                    'total_detecciones': len(labels),
                    'clases_detectadas': todas_clases,
                    'estadisticas_persona': {
                        'numero_instancias': personas_detectadas,
                        'scores_individuales': scores_personas,
                        'score_promedio': np.mean(scores_personas) if scores_personas else 0.0,
                        'score_maximo_persona': max(scores_personas) if scores_personas else 0.0
                    },
                    'analisis_mascaras_avanzado': analisis_mascaras,
                    'confianza_scores': scores_personas,
                    'score_maximo': max(todos_scores) if todos_scores else 0.0,
                    'todas_detecciones_scores': todos_scores
                }

            except Exception as e:
                self.logger.error(f"Error en umbral {umbral}: {str(e)}")
                resultados_umbrales[f'umbral_{umbral}'] = self._crear_resultado_error_umbral(str(e))

        return {'detecciones_por_umbral': resultados_umbrales}

    def _crear_resultado_error(self, error_msg: str, umbrales: List[float]) -> Dict:
        """Crea estructura de resultado para casos de error"""
        resultados_umbrales = {}
        for umbral in umbrales:
            resultados_umbrales[f'umbral_{umbral}'] = self._crear_resultado_error_umbral(error_msg)

        return {
            'error_general': error_msg,
            'detecciones_por_umbral': resultados_umbrales,
            'rendimiento': {'tiempo_inferencia_ms': 0, 'error': True}
        }

    def _crear_resultado_error_umbral(self, error_msg: str) -> Dict:
        """Crea resultado de error para un umbral específico"""
        return {
            'error': error_msg,
            'personas_detectadas': 0,
            'total_detecciones': 0,
            'confianza_scores': [],
            'score_maximo': 0.0,
            'analisis_mascaras_avanzado': None
        }

    def liberar_memoria(self) -> None:
        """Libera recursos del modelo"""
        self.logger.info(f"🧹 Liberando memoria del modelo {self.modelo_info.nombre_corto}")

        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'processor'):
            del self.processor

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        self.logger.info("✅ Memoria liberada")


In [113]:
# =============================================================================
# PROCESADOR DE RESULTADOS AVANZADO
# =============================================================================

class ProcesadorResultadosAvanzado:
    """Procesador completo con análisis avanzado y exportación múltiple"""

    def __init__(self, directorio_salida: Path, logger_manager: LoggerManager):
        self.directorio_salida = directorio_salida
        self.logger_manager = logger_manager
        self.logger = logger_manager.crear_logger("procesador")

        # Inicializar componentes especializados
        self.extractor = ExtractorCaracteristicasAvanzado()
        self.extractor.set_logger(self.logger)
        self.mask_analyzer = MaskAnalyzer()

        # Inicializar generador de visualizaciones
        self.generador_vis = GeneradorVisualizaciones(
            directorio_salida / "visualizaciones",
            self.logger
        )

        # Contadores
        self.imagenes_procesadas = 0
        self.imagenes_exitosas = 0
        self.tiempo_total_procesamiento = 0

        # Lista para guardar resultados y generar resumen visual al final
        self.resultados_para_resumen = []

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                       umbrales: List[float], exportador_coco: ExportadorCOCO = None) -> Optional[Dict[str, Any]]:
        """Procesa una imagen con análisis completo y visualizaciones"""
        inicio_procesamiento = time.time()
        nombre_archivo = os.path.basename(ruta_imagen)

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_imagen = Utils.calcular_hash_imagen(ruta_imagen)

            # Análisis avanzado de características de imagen
            self.logger.info(f"📷 Extrayendo características avanzadas de {nombre_archivo}")
            caracteristicas_avanzadas = self.extractor.analizar_imagen_completa(imagen, ruta_imagen)

            # Información completa de la imagen
            info_imagen = {
                'archivo': nombre_archivo,
                'ruta_completa': ruta_imagen,
                'hash_md5': hash_imagen,
                'caracteristicas_avanzadas': caracteristicas_avanzadas,
                'timestamp_procesamiento': datetime.now().isoformat()
            }

            # Detección de personas con análisis de máscaras
            self.logger.info(f"🔍 Ejecutando detección de personas en {nombre_archivo}")
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            # GENERAR VISUALIZACIÓN INMEDIATAMENTE
            self.logger.info(f"🎨 Generando visualización para {nombre_archivo}")
            archivo_visualizacion = self.generador_vis.generar_visualizacion_completa(
                imagen, nombre_archivo, resultados_deteccion, umbrales[0],
                detector.modelo_info.nombre_corto
            )

            # Exportar a COCO si se especifica
            if exportador_coco and resultados_deteccion.get('detecciones_por_umbral'):
                self._exportar_a_coco(exportador_coco, nombre_archivo, imagen.size,
                                    resultados_deteccion, umbrales[0])

            # Calcular métricas de procesamiento
            tiempo_total_ms = (time.time() - inicio_procesamiento) * 1000

            resultado_completo = {
                'metadatos': {
                    'timestamp': datetime.now().isoformat(),
                    'version_framework': '2.0_avanzado',
                    'librerias_usadas': ['mahotas', 'scikit-image', 'shapely', 'sklearn'],
                    'exitoso': True
                },
                'imagen': info_imagen,
                'deteccion': resultados_deteccion,
                'visualizacion': {
                    'archivo_generado': archivo_visualizacion,
                    'disponible': archivo_visualizacion is not None
                },
                'rendimiento_total': {
                    'tiempo_procesamiento_completo_ms': tiempo_total_ms,
                    'memoria_sistema_mb': self._obtener_memoria_sistema()
                }
            }

            # Estadísticas de logging
            self._log_resultado_imagen(resultado_completo, nombre_archivo)

            # Guardar para resumen visual posterior
            self.resultados_para_resumen.append(resultado_completo)

            # Actualizar contadores
            self.imagenes_procesadas += 1
            self.imagenes_exitosas += 1
            self.tiempo_total_procesamiento += tiempo_total_ms

            return resultado_completo

        except Exception as e:
            self.logger.error(f"❌ Error procesando {nombre_archivo}: {str(e)}")
            self.logger_manager.log_error("procesador", e, f"Imagen: {nombre_archivo}")

            resultado_error = {
                'metadatos': {
                    'timestamp': datetime.now().isoformat(),
                    'version_framework': '2.0_avanzado',
                    'exitoso': False
                },
                'imagen': {'archivo': nombre_archivo, 'error': str(e)},
                'error_detalle': str(e),
                'visualizacion': {
                    'archivo_generado': None,
                    'disponible': False
                }
            }

            self.imagenes_procesadas += 1
            return resultado_error

    def finalizar_procesamiento(self, nombre_modelo: str) -> Optional[str]:
        """Genera resumen visual final y limpia recursos"""
        try:
            # Generar resumen visual con todas las imágenes procesadas
            self.logger.info("🎨 Generando resumen visual final...")
            archivo_resumen_visual = self.generador_vis.generar_resumen_visual(
                self.resultados_para_resumen, nombre_modelo
            )

            # Limpiar lista para siguiente modelo
            self.resultados_para_resumen = []

            return archivo_resumen_visual

        except Exception as e:
            self.logger.error(f"Error generando resumen visual final: {str(e)}")
            return None

    def _exportar_a_coco(self, exportador_coco: ExportadorCOCO, nombre_archivo: str,
                        tamaño_imagen: Tuple[int, int], resultados_deteccion: Dict, umbral_ref: float):
        """Exporta detecciones al formato COCO"""
        try:
            ancho, alto = tamaño_imagen
            image_id = exportador_coco.agregar_imagen(nombre_archivo, ancho, alto)

            detecciones_umbral = resultados_deteccion.get('detecciones_por_umbral', {})
            datos_umbral = detecciones_umbral.get(f'umbral_{umbral_ref}', {})

            # Exportar máscaras individuales si están disponibles
            analisis_mascaras = datos_umbral.get('analisis_mascaras_avanzado')
            if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
                for mascara_data in analisis_mascaras['mascaras_individuales']:
                    if 'error' not in mascara_data:
                        geom = mascara_data.get('caracteristicas_geometricas', {})
                        score = mascara_data.get('score_confianza', 0.0)

                        # Crear bbox desde las características geométricas
                        if all(key in geom for key in ['bbox_min_col', 'bbox_min_row', 'bbox_max_col', 'bbox_max_row']):
                            x = int(geom['bbox_min_col'])
                            y = int(geom['bbox_min_row'])
                            w = int(geom['bbox_max_col'] - geom['bbox_min_col'])
                            h = int(geom['bbox_max_row'] - geom['bbox_min_row'])
                            bbox = [x, y, w, h]

                            # Crear máscara dummy para COCO (en implementación real usarías la máscara real)
                            mascara_dummy = np.zeros((alto, ancho))
                            if x + w <= ancho and y + h <= alto:
                                mascara_dummy[y:y+h, x:x+w] = 1

                            exportador_coco.agregar_anotacion(image_id, mascara_dummy, score, bbox)

        except Exception as e:
            self.logger.warning(f"Error exportando a COCO para {nombre_archivo}: {str(e)}")

    def _obtener_memoria_sistema(self) -> float:
        """Obtiene el uso actual de memoria del sistema"""
        try:
            return psutil.Process().memory_info().rss / (1024 ** 2)
        except:
            return 0.0

    def _log_resultado_imagen(self, resultado: Dict, nombre_archivo: str) -> None:
        """Registra estadísticas del resultado de una imagen"""
        if resultado['metadatos']['exitoso']:
            deteccion = resultado['deteccion']
            rendimiento = deteccion.get('rendimiento', {})

            # Tomar el primer umbral como referencia para logging
            primera_deteccion = None
            for key, value in deteccion.get('detecciones_por_umbral', {}).items():
                primera_deteccion = value
                break

            if primera_deteccion and 'error' not in primera_deteccion:
                personas = primera_deteccion.get('personas_detectadas', 0)
                total = primera_deteccion.get('total_detecciones', 0)
                tiempo = rendimiento.get('tiempo_inferencia_ms', 0)

                self.logger.info(f"✅ {nombre_archivo}: {personas} personas | {total} total | {tiempo:.1f}ms | Análisis avanzado ✓")

                # Log características avanzadas si están disponibles
                carac_avanzadas = resultado['imagen'].get('caracteristicas_avanzadas', {})
                if carac_avanzadas:
                    texturas_mh = carac_avanzadas.get('texturas_mahotas', {})
                    if 'haralick_features' in texturas_mh and texturas_mh['haralick_features']:
                        haralick_mean = np.mean(texturas_mh['haralick_features'])
                        self.logger.info(f"   📊 Haralick promedio: {haralick_mean:.3f}")

                    geom = carac_avanzadas.get('caracteristicas_geometricas', {})
                    if 'num_corners' in geom:
                        self.logger.info(f"   🔍 Esquinas detectadas: {geom['num_corners']}")
        else:
            self.logger.error(f"❌ {nombre_archivo}: Procesamiento fallido")

    def generar_resumen_estadistico_avanzado(self, resultados: List[Dict], config_umbral: ConfiguracionUmbrales,
                                           modelo_info: ModeloInfo) -> Dict[str, Any]:
        """Genera resumen estadístico completo con análisis avanzado"""
        exitosos = [r for r in resultados if r.get('metadatos', {}).get('exitoso', False)]

        resumen = {
            'metadatos_resumen': {
                'timestamp': datetime.now().isoformat(),
                'modelo_evaluado': asdict(modelo_info),
                'configuracion_umbrales': asdict(config_umbral),
                'total_imagenes': len(resultados),
                'imagenes_exitosas': len(exitosos),
                'tasa_exito': len(exitosos) / len(resultados) if resultados else 0,
                'version_framework': '2.0_avanzado'
            },
            'estadisticas_rendimiento': self._calcular_estadisticas_rendimiento(exitosos),
            'estadisticas_deteccion': self._calcular_estadisticas_deteccion(exitosos, config_umbral.valores),
            'estadisticas_caracteristicas_avanzadas': self._analizar_caracteristicas_avanzadas(exitosos),
            'estadisticas_mascaras_shapely': self._analizar_estadisticas_shapely(exitosos),
            'distribucion_caracteristicas': self._analizar_caracteristicas_imagenes(exitosos)
        }

        return resumen

    def _calcular_estadisticas_rendimiento(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Calcula estadísticas de rendimiento del sistema"""
        if not resultados_exitosos:
            return {}

        tiempos_inferencia = []
        memoria_usada = []

        for resultado in resultados_exitosos:
            rendimiento = resultado.get('deteccion', {}).get('rendimiento', {})
            if 'tiempo_inferencia_ms' in rendimiento:
                tiempos_inferencia.append(rendimiento['tiempo_inferencia_ms'])
            if 'memoria_usada_mb' in rendimiento:
                memoria_usada.append(rendimiento['memoria_usada_mb'])

        estadisticas = {
            'tiempo_inferencia': {
                'promedio_ms': np.mean(tiempos_inferencia) if tiempos_inferencia else 0,
                'mediana_ms': np.median(tiempos_inferencia) if tiempos_inferencia else 0,
                'minimo_ms': np.min(tiempos_inferencia) if tiempos_inferencia else 0,
                'maximo_ms': np.max(tiempos_inferencia) if tiempos_inferencia else 0,
                'desviacion_std': np.std(tiempos_inferencia) if tiempos_inferencia else 0
            }
        }

        if memoria_usada:
            estadisticas['memoria'] = {
                'promedio_mb': np.mean(memoria_usada),
                'maxima_mb': np.max(memoria_usada),
                'minima_mb': np.min(memoria_usada)
            }

        return estadisticas

    def _calcular_estadisticas_deteccion(self, resultados_exitosos: List[Dict], umbrales: List[float]) -> Dict[str, Any]:
        """Calcula estadísticas de detección por umbral"""
        if not resultados_exitosos:
            return {}

        estadisticas_por_umbral = {}

        for umbral in umbrales:
            personas_detectadas = []
            scores_maximos = []

            for resultado in resultados_exitosos:
                detecciones = resultado.get('deteccion', {}).get('detecciones_por_umbral', {})
                umbral_data = detecciones.get(f'umbral_{umbral}', {})

                if 'error' not in umbral_data:
                    personas_detectadas.append(umbral_data.get('personas_detectadas', 0))
                    scores_maximos.append(umbral_data.get('score_maximo', 0))

            if personas_detectadas:
                total_personas = sum(personas_detectadas)
                imagenes_con_personas = sum(1 for p in personas_detectadas if p > 0)

                estadisticas_por_umbral[f'umbral_{umbral}'] = {
                    'total_personas_detectadas': total_personas,
                    'imagenes_con_personas': imagenes_con_personas,
                    'imagenes_sin_personas': len(personas_detectadas) - imagenes_con_personas,
                    'porcentaje_imagenes_con_personas': (imagenes_con_personas / len(personas_detectadas)) * 100,
                    'personas_promedio_por_imagen': total_personas / len(personas_detectadas),
                    'score_promedio': np.mean(scores_maximos) if scores_maximos else 0,
                    'score_mediano': np.median(scores_maximos) if scores_maximos else 0
                }

        return estadisticas_por_umbral

    def _analizar_caracteristicas_avanzadas(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza las características avanzadas extraídas"""
        if not resultados_exitosos:
            return {}

        haralick_means = []
        glcm_contrasts = []
        num_corners = []
        zernike_means = []
        num_regiones = []

        for resultado in resultados_exitosos:
            carac = resultado.get('imagen', {}).get('caracteristicas_avanzadas', {})

            # Características Haralick
            texturas_mh = carac.get('texturas_mahotas', {})
            if 'haralick_features' in texturas_mh and texturas_mh['haralick_features']:
                haralick_means.append(np.mean(texturas_mh['haralick_features']))

            # Zernike moments
            if 'zernike_moments' in texturas_mh and texturas_mh['zernike_moments']:
                zernike_means.append(np.mean(texturas_mh['zernike_moments']))

            # GLCM contrast
            texturas_sk = carac.get('texturas_skimage', {})
            glcm_props = texturas_sk.get('glcm_properties', {})
            if 'contrast' in glcm_props:
                glcm_contrasts.append(glcm_props['contrast'])

            # Características geométricas
            geom = carac.get('caracteristicas_geometricas', {})
            if 'num_corners' in geom:
                num_corners.append(geom['num_corners'])

            # Regiones
            regionales = carac.get('propiedades_regionales', {})
            if 'num_regiones' in regionales:
                num_regiones.append(regionales['num_regiones'])

        def safe_stats(values, nombre):
            if not values:
                return {'count': 0, 'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'count': len(values),
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'haralick_estadisticas': safe_stats(haralick_means, 'Haralick'),
            'glcm_contrast_estadisticas': safe_stats(glcm_contrasts, 'GLCM Contrast'),
            'corners_estadisticas': safe_stats(num_corners, 'Corners'),
            'zernike_estadisticas': safe_stats(zernike_means, 'Zernike'),
            'regiones_estadisticas': safe_stats(num_regiones, 'Regiones'),
            'imagenes_con_caracteristicas': len([r for r in resultados_exitosos
                                               if r.get('imagen', {}).get('caracteristicas_avanzadas')])
        }

    def _analizar_estadisticas_shapely(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza estadísticas específicas de análisis con Shapely"""
        if not resultados_exitosos:
            return {}

        compactness_values = []
        convexity_ratios = []
        roughness_factors = []
        num_vertices = []
        mascaras_validas = 0
        total_mascaras = 0

        for resultado in resultados_exitosos:
            detecciones = resultado.get('deteccion', {}).get('detecciones_por_umbral', {})

            for umbral_key, umbral_data in detecciones.items():
                analisis_mascaras = umbral_data.get('analisis_mascaras_avanzado')
                if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
                    for mascara_data in analisis_mascaras['mascaras_individuales']:
                        total_mascaras += 1

                        if 'error' not in mascara_data:
                            # Características geométricas
                            geom = mascara_data.get('caracteristicas_geometricas', {})
                            if 'compactness' in geom:
                                compactness_values.append(geom['compactness'])

                            # Características Shapely
                            shapely_geom = mascara_data.get('shapely_geometry', {})
                            if shapely_geom.get('polygon_valido'):
                                mascaras_validas += 1
                                if 'convexity_ratio' in shapely_geom:
                                    convexity_ratios.append(shapely_geom['convexity_ratio'])
                                if 'num_vertices' in shapely_geom:
                                    num_vertices.append(shapely_geom['num_vertices'])

                            # Características de forma
                            forma = mascara_data.get('caracteristicas_forma', {})
                            if 'roughness_factor' in forma:
                                roughness_factors.append(forma['roughness_factor'])

        def safe_stats(values):
            if not values:
                return {'count': 0, 'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'count': len(values),
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'total_mascaras_analizadas': total_mascaras,
            'mascaras_shapely_validas': mascaras_validas,
            'tasa_validez_shapely': mascaras_validas / total_mascaras if total_mascaras > 0 else 0,
            'compactness_estadisticas': safe_stats(compactness_values),
            'convexity_ratio_estadisticas': safe_stats(convexity_ratios),
            'roughness_estadisticas': safe_stats(roughness_factors),
            'vertices_estadisticas': safe_stats(num_vertices)
        }

    def _analizar_caracteristicas_imagenes(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza la distribución de características de las imágenes procesadas"""
        if not resultados_exitosos:
            return {}

        anchos, altos, brillos, contrastes = [], [], [], []
        megapixeles, aspect_ratios = [], []

        for resultado in resultados_exitosos:
            carac_avanzadas = resultado.get('imagen', {}).get('caracteristicas_avanzadas', {})

            # Metadatos básicos
            metadatos = carac_avanzadas.get('metadatos_basicos', {})
            if metadatos:
                dims = metadatos.get('dimensiones', {})
                if dims:
                    anchos.append(dims.get('ancho', 0))
                    altos.append(dims.get('alto', 0))

                if 'megapixeles' in metadatos:
                    megapixeles.append(metadatos['megapixeles'])
                if 'aspecto_ratio' in metadatos:
                    aspect_ratios.append(metadatos['aspecto_ratio'])

            # Características de color y luminancia
            color = carac_avanzadas.get('color_y_paleta', {})
            if color:
                lab = color.get('lab_luminancia', {})
                if 'L_medio' in lab:
                    brillos.append(lab['L_medio'])

                rgb_stats = color.get('estadisticas_rgb', {})
                if 'std' in rgb_stats and rgb_stats['std']:
                    # Promedio de desviación estándar RGB como medida de contraste
                    contrastes.append(np.mean(rgb_stats['std']))

        def safe_stats(values):
            if not values:
                return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'dimensiones': {
                'ancho_estadisticas': safe_stats(anchos),
                'alto_estadisticas': safe_stats(altos),
                'megapixeles_estadisticas': safe_stats(megapixeles),
                'aspect_ratio_estadisticas': safe_stats(aspect_ratios)
            },
            'iluminacion_color': {
                'brillo_lab_estadisticas': safe_stats(brillos),
                'contraste_rgb_estadisticas': safe_stats(contrastes)
            },
            'resumen_dataset': {
                'imagenes_analizadas': len(resultados_exitosos),
                'resolucion_promedio_mp': np.mean(megapixeles) if megapixeles else 0,
                'orientacion_predominante': 'horizontal' if np.mean(aspect_ratios) > 1 else 'vertical' if aspect_ratios else 'desconocida'
            }
        }

    def mostrar_resumen_consola_avanzado(self, resumen: Dict[str, Any]) -> None:
        """Muestra un resumen completo en consola con características avanzadas"""
        meta = resumen['metadatos_resumen']
        modelo = meta['modelo_evaluado']

        print(f"\n{'='*100}")
        print(f"📊 RESUMEN DE EVALUACIÓN AVANZADA")
        print(f"{'='*100}")
        print(f"🤖 Modelo: {modelo['nombre_corto']}")
        print(f"📊 Dataset: {modelo['dataset']} | Tipo: {modelo['tipo'].upper()}")
        print(f"✅ Éxito: {meta['imagenes_exitosas']}/{meta['total_imagenes']} ({meta['tasa_exito']*100:.1f}%)")
        print(f"🔬 Framework: {meta['version_framework']}")

        # Estadísticas de rendimiento
        rendimiento = resumen.get('estadisticas_rendimiento', {})
        if 'tiempo_inferencia' in rendimiento:
            tiempo = rendimiento['tiempo_inferencia']
            print(f"⚡ Tiempo promedio: {tiempo['promedio_ms']:.1f}ms (min: {tiempo['minimo_ms']:.1f}ms, max: {tiempo['maximo_ms']:.1f}ms)")

        # Estadísticas por umbral
        deteccion = resumen.get('estadisticas_deteccion', {})
        if deteccion:
            print(f"\n🎯 RESULTADOS POR UMBRAL:")
            for umbral_key, stats in deteccion.items():
                umbral_val = umbral_key.replace('umbral_', '')
                print(f"   {umbral_val:>8}: {stats['imagenes_con_personas']:3d}/{meta['imagenes_exitosas']} imágenes ({stats['porcentaje_imagenes_con_personas']:5.1f}%) | {stats['total_personas_detectadas']:3d} personas")

        # Características avanzadas
        carac_avanzadas = resumen.get('estadisticas_caracteristicas_avanzadas', {})
        if carac_avanzadas and carac_avanzadas.get('imagenes_con_caracteristicas', 0) > 0:
            print(f"\n🔬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:")

            haralick = carac_avanzadas.get('haralick_estadisticas', {})
            if haralick.get('count', 0) > 0:
                print(f"   📊 Haralick (textura): {haralick['mean']:.3f} ± {haralick['std']:.3f}")

            corners = carac_avanzadas.get('corners_estadisticas', {})
            if corners.get('count', 0) > 0:
                print(f"   🔍 Esquinas detectadas: {corners['mean']:.1f} ± {corners['std']:.1f}")

            glcm = carac_avanzadas.get('glcm_contrast_estadisticas', {})
            if glcm.get('count', 0) > 0:
                print(f"   ⚡ GLCM Contraste: {glcm['mean']:.3f} ± {glcm['std']:.3f}")

        # Estadísticas Shapely
        shapely_stats = resumen.get('estadisticas_mascaras_shapely', {})
        if shapely_stats and shapely_stats.get('total_mascaras_analizadas', 0) > 0:
            print(f"\n🔷 ANÁLISIS GEOMÉTRICO SHAPELY:")
            print(f"   📐 Máscaras analizadas: {shapely_stats['total_mascaras_analizadas']}")
            print(f"   ✅ Polígonos válidos: {shapely_stats['mascaras_shapely_validas']} ({shapely_stats['tasa_validez_shapely']*100:.1f}%)")

            compactness = shapely_stats.get('compactness_estadisticas', {})
            if compactness.get('count', 0) > 0:
                print(f"   🔄 Compacidad promedio: {compactness['mean']:.3f} ± {compactness['std']:.3f}")

            convexity = shapely_stats.get('convexity_ratio_estadisticas', {})
            if convexity.get('count', 0) > 0:
                print(f"   📏 Ratio convexidad: {convexity['mean']:.3f} ± {convexity['std']:.3f}")

In [114]:
# =============================================================================
# EVALUADOR PRINCIPAL AVANZADO
# =============================================================================

class EvaluadorMask2FormerAvanzado:
    """Clase principal que orquesta todo el proceso de evaluación avanzado"""

    def __init__(self, config: ConfigEvaluacion):
        self.config = config

        # Crear directorio de ejecución
        self.directorio_ejecucion = config.crear_directorio_ejecucion()

        # Inicializar sistema de logging
        self.logger_manager = LoggerManager(self.directorio_ejecucion / "logs")
        self.logger = self.logger_manager.crear_logger("evaluador_principal")

        # Inicializar procesador avanzado
        self.procesador = ProcesadorResultadosAvanzado(self.directorio_ejecucion, self.logger_manager)

        # Log de inicialización
        self.logger.info("🚀 EVALUADOR MASK2FORMER AVANZADO INICIALIZADO")
        self.logger.info(f"📁 Directorio de ejecución: {self.directorio_ejecucion}")
        self.logger.info(f"🎯 Modelos disponibles: {len(self.config.MODELOS)}")
        self.logger.info(f"⚙️ Configuraciones de umbral: {len(self.config.UMBRALES)}")
        self.logger.info(f"🔬 Análisis avanzado: Mahotas + Scikit-image + Shapely + OpenCV")

    def ejecutar_evaluacion_modelo(self, idx_modelo: int, nombre_config_umbral: str) -> Optional[str]:
        """Ejecuta evaluación completa para un modelo y configuración específica"""
        # Validaciones
        if idx_modelo >= len(self.config.MODELOS):
            self.logger.error(f"❌ Índice de modelo inválido: {idx_modelo}")
            return None

        if nombre_config_umbral not in self.config.UMBRALES:
            self.logger.error(f"❌ Configuración de umbral inválida: {nombre_config_umbral}")
            return None

        # Configuración actual
        modelo_info = self.config.MODELOS[idx_modelo]
        config_umbral = self.config.UMBRALES[nombre_config_umbral]

        # Log de inicio
        self.logger.info("="*100)
        self.logger.info(f"🚀 INICIANDO EVALUACIÓN AVANZADA")
        self.logger.info("="*100)
        self.logger.info(f"🤖 Modelo: {modelo_info.nombre_corto}")
        self.logger.info(f"⚙️ Umbrales: {config_umbral.descripcion}")
        self.logger.info(f"🎯 Valores: {config_umbral.valores}")

        try:
            # Cargar dataset
            imagenes = Utils.cargar_imagenes(str(self.config.DATASET_PATH))
            if not imagenes:
                self.logger.error("❌ No se encontraron imágenes en el dataset")
                return None

            self.logger.info(f"📸 Imágenes encontradas: {len(imagenes)}")

            # Limitar imágenes si es necesario
            if len(imagenes) > self.config.MAX_IMAGENES_LOTE:
                imagenes = imagenes[:self.config.MAX_IMAGENES_LOTE]
                self.logger.info(f"⚠️ Limitando a {self.config.MAX_IMAGENES_LOTE} imágenes para prueba")

            # Inicializar detector con componentes avanzados
            detector_logger = self.logger_manager.crear_logger(f"detector_{modelo_info.nombre_corto}")
            detector = DetectorPersonas(
                modelo_info,
                detector_logger,
                self.procesador.extractor,
                self.procesador.mask_analyzer
            )

            # Inicializar exportador COCO si está habilitado
            exportador_coco = None
            if self.config.GENERAR_FORMATO_COCO:
                exportador_coco = ExportadorCOCO(self.logger)

            # Procesar imágenes
            self.logger.info(f"🔄 INICIANDO PROCESAMIENTO AVANZADO DE {len(imagenes)} IMÁGENES")
            resultados = []
            tiempo_inicio_total = time.time()

            for i, ruta_imagen in enumerate(tqdm(imagenes, desc="Procesando con análisis avanzado"), 1):
                # Limpiar caché periódicamente
                if i % self.config.LIMPIAR_CACHE_CADA == 0:
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

                resultado = self.procesador.procesar_imagen(
                    ruta_imagen, detector, config_umbral.valores, exportador_coco
                )

                if resultado:
                    resultados.append(resultado)

                # Log de progreso
                if i % 10 == 0 or i == len(imagenes):
                    self.logger.info(f"📈 Progreso: {i}/{len(imagenes)} imágenes procesadas")

            tiempo_total_s = time.time() - tiempo_inicio_total

            # Generar resumen estadístico avanzado
            resumen = self.procesador.generar_resumen_estadistico_avanzado(
                resultados, config_umbral, modelo_info
            )

            # Generar resumen visual final
            archivo_resumen_visual = self.procesador.finalizar_procesamiento(modelo_info.nombre_corto)

            # Guardar resultados en múltiples formatos
            archivos_generados = self._guardar_resultados_multiples_formatos(
                resultados, resumen, modelo_info, config_umbral, exportador_coco, archivo_resumen_visual
            )

            # Mostrar resumen en consola
            self.procesador.mostrar_resumen_consola_avanzado(resumen)

            # Log final
            self.logger.info(f"⏱️ Tiempo total: {tiempo_total_s:.1f} segundos")
            self.logger.info(f"💾 Archivos generados: {len(archivos_generados)}")
            for formato, archivo in archivos_generados.items():
                self.logger.info(f"   📄 {formato}: {Path(archivo).name}")

            # Liberar memoria del detector
            detector.liberar_memoria()

            return archivos_generados.get('json_principal')

        except Exception as e:
            self.logger.error(f"❌ Error durante evaluación: {str(e)}")
            self.logger_manager.log_error("evaluador_principal", e, "Evaluación modelo")
            return None

    def _guardar_resultados_multiples_formatos(self, resultados: List[Dict], resumen: Dict,
                                             modelo_info: ModeloInfo, config_umbral: ConfiguracionUmbrales,
                                             exportador_coco: ExportadorCOCO = None,
                                             archivo_resumen_visual: str = None) -> Dict[str, str]:
        """Guarda resultados en múltiples formatos"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_base = Utils.crear_nombre_archivo(modelo_info, config_umbral.nombre, timestamp)

        archivos_generados = {}

        # 1. Archivo JSON principal con análisis completo
        datos_completos = {
            'metadatos_archivo': {
                'timestamp_creacion': datetime.now().isoformat(),
                'version_framework': '2.0_avanzado',
                'modelo_evaluado': asdict(modelo_info),
                'configuracion_umbrales': asdict(config_umbral),
                'directorio_ejecucion': str(self.directorio_ejecucion),
                'total_imagenes_procesadas': len(resultados),
                'librerias_usadas': ['mahotas', 'scikit-image', 'shapely', 'sklearn', 'opencv-python'],
                'analisis_incluidos': [
                    'caracteristicas_haralick',
                    'texturas_lbp_glcm',
                    'geometria_shapely',
                    'descriptores_locales',
                    'analisis_multiscala',
                    'segmentacion_regional'
                ]
            },
            'resumen_estadistico': resumen,
            'resultados_detallados': resultados
        }

        archivo_principal = self.directorio_ejecucion / "resultados_json" / nombre_base
        Utils.guardar_json(datos_completos, archivo_principal)
        archivos_generados['json_principal'] = str(archivo_principal)

        # 2. Resumen ejecutivo separado
        archivo_resumen = self.directorio_ejecucion / "resumenes" / f"resumen_{nombre_base}"
        Utils.guardar_json(resumen, archivo_resumen)
        archivos_generados['resumen_ejecutivo'] = str(archivo_resumen)

        # 3. Características avanzadas separadas
        caracteristicas_separadas = {
            'metadatos': {
                'timestamp': datetime.now().isoformat(),
                'modelo': asdict(modelo_info),
                'descripcion': 'Características avanzadas extraídas con librerías especializadas'
            },
            'caracteristicas_por_imagen': []
        }

        for resultado in resultados:
            if resultado.get('metadatos', {}).get('exitoso'):
                carac_img = {
                    'archivo': resultado['imagen']['archivo'],
                    'caracteristicas_avanzadas': resultado['imagen'].get('caracteristicas_avanzadas', {})
                }
                caracteristicas_separadas['caracteristicas_por_imagen'].append(carac_img)

        archivo_caracteristicas = self.directorio_ejecucion / "caracteristicas_avanzadas" / f"caracteristicas_{nombre_base}"
        Utils.guardar_json(caracteristicas_separadas, archivo_caracteristicas)
        archivos_generados['caracteristicas_avanzadas'] = str(archivo_caracteristicas)

        # 4. Formato COCO si está habilitado
        if exportador_coco and self.config.GENERAR_FORMATO_COCO:
            nombre_coco = nombre_base.replace('.json', '_coco.json')
            archivo_coco = self.directorio_ejecucion / "formato_coco" / nombre_coco
            exportador_coco.exportar(archivo_coco)
            archivos_generados['formato_coco'] = str(archivo_coco)

        # 5. Resumen visual si está disponible
        if archivo_resumen_visual:
            archivos_generados['resumen_visual'] = archivo_resumen_visual

        # Log de archivos generados
        self.logger.info(f"💾 Archivo principal: {Path(archivo_principal).name}")
        self.logger.info(f"💾 Resumen ejecutivo: {Path(archivo_resumen).name}")
        self.logger.info(f"💾 Características avanzadas: {Path(archivo_caracteristicas).name}")
        if 'formato_coco' in archivos_generados:
            self.logger.info(f"💾 Formato COCO: {Path(archivos_generados['formato_coco']).name}")
        if 'resumen_visual' in archivos_generados:
            self.logger.info(f"🎨 Resumen visual: {Path(archivos_generados['resumen_visual']).name}")

        return archivos_generados

    def ejecutar_evaluacion_completa(self) -> Dict[str, List[str]]:
        """Ejecuta evaluación completa de todos los modelos con todas las configuraciones"""
        total_combinaciones = len(self.config.MODELOS) * len(self.config.UMBRALES)
        archivos_generados = {}

        self.logger.info(f"🎯 EVALUACIÓN COMPLETA AVANZADA: {total_combinaciones} combinaciones")
        self.logger.info(f"🔬 Análisis incluidos: Texturas, Geometría, Color, Forma, Multi-escala")

        combinacion_actual = 0

        try:
            for i, modelo_info in enumerate(self.config.MODELOS):
                archivos_modelo = []

                for nombre_config in self.config.UMBRALES.keys():
                    combinacion_actual += 1

                    self.logger.info(f"\n{'='*100}")
                    self.logger.info(f"📊 COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                    self.logger.info(f"{'='*100}")

                    archivo_resultado = self.ejecutar_evaluacion_modelo(i, nombre_config)

                    if archivo_resultado:
                        archivos_modelo.append(archivo_resultado)
                        self.logger.info(f"✅ Combinación {combinacion_actual} completada exitosamente")
                    else:
                        self.logger.error(f"❌ Error en combinación {combinacion_actual}")

                    # Pausa entre combinaciones
                    if combinacion_actual < total_combinaciones:
                        self.logger.info("⏳ Pausa de 3 segundos...")
                        time.sleep(3)

                archivos_generados[modelo_info.nombre_corto] = archivos_modelo

            self.logger.info(f"\n🎉 EVALUACIÓN COMPLETA AVANZADA FINALIZADA")
            self.logger.info(f"📁 Todos los resultados en: {self.directorio_ejecucion}")
            self.logger.info(f"🔬 Análisis completo con librerías especializadas")

            return archivos_generados

        except KeyboardInterrupt:
            self.logger.warning(f"\n⚠️ EVALUACIÓN INTERRUMPIDA POR USUARIO")
            self.logger.info(f"📊 Progreso: {combinacion_actual}/{total_combinaciones}")
            return archivos_generados

        except Exception as e:
            self.logger.error(f"\n❌ ERROR CRÍTICO: {str(e)}")
            self.logger_manager.log_error("evaluador_principal", e, "Evaluación completa")
            return archivos_generados


In [115]:
# =============================================================================
# FUNCIÓN PRINCIPAL UNIFICADA
# =============================================================================

def main():
    """
    Función principal única del sistema de evaluación avanzado
    Ejecuta automáticamente todos los modelos con todas las configuraciones
    """
    try:
        # Configuración inicial
        config = ConfigEvaluacion()

        print(f"\n{'='*100}")
        print(f"🎯 SISTEMA DE EVALUACIÓN MASK2FORMER AVANZADO - TFM NIVEL MÁSTER")
        print(f"{'='*100}")
        print(f"🔬 Análisis incluidos:")
        print(f"   📊 Texturas: Haralick, LBP, GLCM, Zernike")
        print(f"   🔷 Geometría: Shapely, Compacidad, Convexidad")
        print(f"   🎨 Color: Paleta dominante, HSV, LAB")
        print(f"   📐 Forma: Bordes, Esquinas, Momentos")
        print(f"   🔍 Multi-escala: Características multi-resolución")
        print(f"   🗺️  Regional: Segmentación SLIC")

        print(f"\n📊 Configuración de evaluación:")
        print(f"   🤖 Modelos: {len(config.MODELOS)}")
        for i, modelo in enumerate(config.MODELOS):
            print(f"      [{i}] {modelo.nombre_corto} ({modelo.tipo}, {modelo.dataset})")

        print(f"   ⚙️ Configuraciones de umbral: {len(config.UMBRALES)}")
        for nombre, config_umbral in config.UMBRALES.items():
            print(f"      {nombre}: {config_umbral.descripcion}")

        print(f"   📄 Formatos de salida: JSON propio + COCO")
        print(f"   🔬 Librerías: Mahotas, Scikit-image, Shapely, OpenCV, Sklearn")

        # Crear evaluador
        evaluador = EvaluadorMask2FormerAvanzado(config)

        # Ejecutar evaluación completa automáticamente
        print(f"\n🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...")
        total_combinaciones = len(config.MODELOS) * len(config.UMBRALES)
        print(f"📊 Total de combinaciones a procesar: {total_combinaciones}")

        archivos_resultado = evaluador.ejecutar_evaluacion_completa()

        # Resumen final
        print(f"\n{'='*100}")
        print(f"✅ EVALUACIÓN COMPLETA FINALIZADA")
        print(f"{'='*100}")

        total_archivos = sum(len(archivos) for archivos in archivos_resultado.values())
        print(f"📁 Archivos generados: {total_archivos}")
        print(f"💾 Ubicación: {evaluador.directorio_ejecucion}")

        print(f"\n📊 RESUMEN POR MODELO:")
        for modelo, archivos in archivos_resultado.items():
            print(f"   🤖 {modelo}: {len(archivos)} evaluaciones completadas")

        print(f"\n📂 ESTRUCTURA DE ARCHIVOS GENERADOS:")
        print(f"   📄 resultados_json/     - Análisis completos JSON")
        print(f"   📊 resumenes/           - Resúmenes ejecutivos")
        print(f"   🔬 caracteristicas_avanzadas/ - Características especializadas")
        print(f"   🏷️  formato_coco/       - Exportación formato COCO")
        print(f"   📝 logs/               - Registros de ejecución")

        print(f"\n🎯 ANÁLISIS COMPLETADO CON ÉXITO")
        print(f"🔬 Framework v2.0 - Análisis avanzado con librerías especializadas")

    except Exception as e:
        print(f"\n❌ ERROR CRÍTICO EN MAIN: {str(e)}")
        import traceback
        traceback.print_exc()
        raise


In [116]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":
    main()

INFO:evaluador_principal:🚀 EVALUADOR MASK2FORMER AVANZADO INICIALIZADO
INFO:evaluador_principal:📁 Directorio de ejecución: /content/drive/MyDrive/TFM/mask2former/resultados/ejecucion_20250927_185350
INFO:evaluador_principal:🎯 Modelos disponibles: 3
INFO:evaluador_principal:⚙️ Configuraciones de umbral: 4
INFO:evaluador_principal:🔬 Análisis avanzado: Mahotas + Scikit-image + Shapely + OpenCV
INFO:evaluador_principal:🎯 EVALUACIÓN COMPLETA AVANZADA: 12 combinaciones
INFO:evaluador_principal:🔬 Análisis incluidos: Texturas, Geometría, Color, Forma, Multi-escala
INFO:evaluador_principal:
INFO:evaluador_principal:📊 COMBINACIÓN 1/12
INFO:evaluador_principal:====================================================================================================
INFO:evaluador_principal:====================================================================================================
INFO:evaluador_principal:🚀 INICIANDO EVALUACIÓN AVANZADA
INFO:evaluador_principal:=================================


🎯 SISTEMA DE EVALUACIÓN MASK2FORMER AVANZADO - TFM NIVEL MÁSTER
🔬 Análisis incluidos:
   📊 Texturas: Haralick, LBP, GLCM, Zernike
   🔷 Geometría: Shapely, Compacidad, Convexidad
   🎨 Color: Paleta dominante, HSV, LAB
   📐 Forma: Bordes, Esquinas, Momentos
   🔍 Multi-escala: Características multi-resolución
   🗺️  Regional: Segmentación SLIC

📊 Configuración de evaluación:
   🤖 Modelos: 3
      [0] swin-large-coco-instance (instancia, COCO)
      [1] swin-base-ade-semantic (semantico, ADE20K)
      [2] swin-small-coco-instance (instancia, COCO)
   ⚙️ Configuraciones de umbral: 4
      ultra_sensible: Detecta cambios mínimos - Máxima sensibilidad
      alta_sensibilidad: Sensibilidad alta para detección temprana
      sensibilidad_media: Balance entre precisión y recall
      baja_sensibilidad: Solo detecciones muy confiables
   📄 Formatos de salida: JSON propio + COCO
   🔬 Librerías: Mahotas, Scikit-image, Shapely, OpenCV, Sklearn

🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...
📊 Total 

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:detector_swin-large-coco-instance:⏳ Cargando modelo de segmentación...
INFO:detector_swin-large-coco-instance:🏷️ Clases disponibles: 80
INFO:detector_swin-large-coco-instance:👤 Clase persona encontrada: ID 0 = 'person'
INFO:detector_swin-large-coco-instance:✅ Modelo cargado exitosamente
INFO:detector_swin-large-coco-instance:============================================================
INFO:evaluador_principal:🔄 INICIANDO PROCESAMIENTO AVANZADO DE 3 IMÁGENES
Procesando con análisis avanzado:   0%|          | 0/3 [00:00<?, ?it/s]INFO:procesador:📷 Extrayendo características avanzadas de Aleksandra_Retrato.jpg
INFO:procesador:🔍 Ejecutando detección de personas en Aleksandra_Retrato.jpg
INFO:procesador:🎨 Generando visualización para Aleksandra_Retrato.jpg
INFO:procesador:Visualización guardada: vis_swin-large-coco-instance_umbral_00001_Aleksandra_Retrato.png
INFO:procesador:✅ Aleksandra_Retrato.jpg: 0 personas | 0 total | 184.9ms | Análisis avanzado ✓
INFO:procesador:   📊 Haralick prom


📊 RESUMEN DE EVALUACIÓN AVANZADA
🤖 Modelo: swin-large-coco-instance
📊 Dataset: COCO | Tipo: INSTANCIA
✅ Éxito: 3/3 (100.0%)
🔬 Framework: 2.0_avanzado
⚡ Tiempo promedio: 186.3ms (min: 182.9ms, max: 191.3ms)

🎯 RESULTADOS POR UMBRAL:
     0.0001:   0/3 imágenes (  0.0%) |   0 personas
      0.001:   0/3 imágenes (  0.0%) |   0 personas
       0.01:   0/3 imágenes (  0.0%) |   0 personas
        0.1:   0/3 imágenes (  0.0%) |   0 personas

🔬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:
   📊 Haralick (textura): 1487.734 ± 344.378
   🔍 Esquinas detectadas: 3725.0 ± 86.2
   ⚡ GLCM Contraste: 54.915 ± 27.816


...


📊 RESUMEN DE EVALUACIÓN AVANZADA
🤖 Modelo: swin-small-coco-instance
📊 Dataset: COCO | Tipo: INSTANCIA
✅ Éxito: 3/3 (100.0%)
🔬 Framework: 2.0_avanzado
⚡ Tiempo promedio: 99.9ms (min: 99.2ms, max: 100.5ms)

🎯 RESULTADOS POR UMBRAL:
       0.01:   0/3 imágenes (  0.0%) |   0 personas
        0.1:   0/3 imágenes (  0.0%) |   0 personas
        0.3:   0/3 imágenes (  0.0%) |   0 personas
        0.5:   0/3 imágenes (  0.0%) |   0 personas

🔬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:
   📊 Haralick (textura): 1487.734 ± 344.378
   🔍 Esquinas detectadas: 3725.0 ± 86.2
   ⚡ GLCM Contraste: 54.915 ± 27.816


INFO:evaluador_principal:
INFO:evaluador_principal:📊 COMBINACIÓN 12/12
INFO:evaluador_principal:====================================================================================================
INFO:evaluador_principal:====================================================================================================
INFO:evaluador_principal:🚀 INICIANDO EVALUACIÓN AVANZADA
INFO:evaluador_principal:====================================================================================================
INFO:evaluador_principal:🤖 Modelo: swin-small-coco-instance
INFO:evaluador_principal:⚙️ Umbrales: Solo detecciones muy confiables
INFO:evaluador_principal:🎯 Valores: [0.3, 0.5, 0.7]
INFO:evaluador_principal:📸 Imágenes encontradas: 3
INFO:detector_swin-small-coco-instance:============================================================
INFO:detector_swin-small-coco-instance:🤖 INICIALIZANDO MODELO: swin-small-coco-instance
INFO:detector_swin-small-coco-instance:📌 Tipo: INSTANCIA
INFO:detector_sw

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:detector_swin-small-coco-instance:⏳ Cargando modelo de segmentación...
INFO:detector_swin-small-coco-instance:🏷️ Clases disponibles: 80
INFO:detector_swin-small-coco-instance:👤 Clase persona encontrada: ID 0 = 'person'
INFO:detector_swin-small-coco-instance:✅ Modelo cargado exitosamente
INFO:detector_swin-small-coco-instance:============================================================
INFO:evaluador_principal:🔄 INICIANDO PROCESAMIENTO AVANZADO DE 3 IMÁGENES
Procesando con análisis avanzado:   0%|          | 0/3 [00:00<?, ?it/s]INFO:procesador:📷 Extrayendo características avanzadas de Aleksandra_Retrato.jpg
INFO:procesador:🔍 Ejecutando detección de personas en Aleksandra_Retrato.jpg
INFO:procesador:🎨 Generando visualización para Aleksandra_Retrato.jpg
INFO:procesador:Visualización guardada: vis_swin-small-coco-instance_umbral_03000_Aleksandra_Retrato.png
INFO:procesador:✅ Aleksandra_Retrato.jpg: 0 personas | 0 total | 108.4ms | Análisis avanzado ✓
INFO:procesador:   📊 Haralick prom


📊 RESUMEN DE EVALUACIÓN AVANZADA
🤖 Modelo: swin-small-coco-instance
📊 Dataset: COCO | Tipo: INSTANCIA
✅ Éxito: 3/3 (100.0%)
🔬 Framework: 2.0_avanzado
⚡ Tiempo promedio: 103.8ms (min: 101.1ms, max: 108.4ms)

🎯 RESULTADOS POR UMBRAL:
        0.3:   0/3 imágenes (  0.0%) |   0 personas
        0.5:   0/3 imágenes (  0.0%) |   0 personas
        0.7:   0/3 imágenes (  0.0%) |   0 personas

🔬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:
   📊 Haralick (textura): 1487.734 ± 344.378
   🔍 Esquinas detectadas: 3725.0 ± 86.2
   ⚡ GLCM Contraste: 54.915 ± 27.816

✅ EVALUACIÓN COMPLETA FINALIZADA
📁 Archivos generados: 12
💾 Ubicación: /content/drive/MyDrive/TFM/mask2former/resultados/ejecucion_20250927_185350

📊 RESUMEN POR MODELO:
   🤖 swin-large-coco-instance: 4 evaluaciones completadas
   🤖 swin-base-ade-semantic: 4 evaluaciones completadas
   🤖 swin-small-coco-instance: 4 evaluaciones completadas

📂 ESTRUCTURA DE ARCHIVOS GENERADOS:
   📄 resultados_json/     - Análisis completos JSON
   📊 resumenes/ 